In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 33.4 MB/s eta 0:00:00


In [ ]:
import re
import html
import json
import urllib.parse
from datetime import datetime, timedelta, timezone

import requests
from bs4 import BeautifulSoup

# ================= НАСТРОЙКИ =================
KEYWORDS = [
    'трамп', 'trump', 'дональд',
    'сша', 'герман', 'китай', 'росси',
    'эконом', 'инфляц', 'ставк', 'ввп', 'moex', 'фрс', 'ecb'
]
DAYS_BACK = 2
MAX_NEWS_PER_SITE = 50           # максимум подходящих новостей на сайт (после фильтров)
MAX_LINKS_TASS = 120             # сколько карточек просматривать у TASS
MAX_LINKS_INTERFAX = 120         # сколько карточек просматривать у Interfax
DEBUG = False                    # включить диагностику отбраковки

# Источники
TASS_LIST_URL = "https://tass.ru/ekonomika"
INTERFAX_LIST_URL = "https://www.interfax.ru/business"

HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/125.0 Safari/537.36"),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
    "Referer": "https://google.com"
}

# Русские месяцы → номер
RU_MONTHS = {
    'января': 1, 'февраля': 2, 'марта': 3, 'апреля': 4, 'мая': 5, 'июня': 6,
    'июля': 7, 'августа': 8, 'сентября': 9, 'октября': 10, 'ноября': 11, 'декабря': 12
}

ISO_REGEX = re.compile(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}(?::\d{2})?(?:Z|[+\-]\d{2}:\d{2})', re.IGNORECASE)

# ================= ВСПОМОГАТЕЛЬНОЕ =================
def now_utc():
    return datetime.now(timezone.utc)

def within_days(dt: datetime, days: int) -> bool:
    """dt должен быть tz-aware; сравнение с текущим UTC."""
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return (now_utc() - dt) <= timedelta(days=days)

def title_matches(title: str) -> bool:
    tl = (title or "").lower()
    return any(kw in tl for kw in KEYWORDS)

def _req(url: str, timeout=20) -> requests.Response | None:
    try:
        r = requests.get(url, headers=HEADERS, timeout=timeout)
        enc = r.encoding or r.apparent_encoding or 'utf-8'
        r.encoding = enc
        if r.status_code == 200:
            return r
        if DEBUG: print(f"[WARN] HTTP {r.status_code} for {url}")
    except Exception as e:
        if DEBUG: print(f"[ERR] {e} for {url}")
    return None

# -------- парсинг дат --------
def parse_iso_or_rfc2822(s: str) -> datetime | None:
    if not s:
        return None
    s = s.strip()
    # ISO 8601
    try:
        if s.endswith('Z'):
            dt = datetime.fromisoformat(s.replace('Z', '+00:00'))
        else:
            dt = datetime.fromisoformat(s)
        return dt.astimezone(timezone.utc)
    except Exception:
        pass
    # RFC2822
    try:
        from email.utils import parsedate_to_datetime
        dt = parsedate_to_datetime(s)
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=timezone.utc)
        return dt.astimezone(timezone.utc)
    except Exception:
        return None

RU_NUMERIC_RE = re.compile(
    r'(?P<d>\d{1,2})\.(?P<m>\d{1,2})\.(?P<y>\d{4})(?:\s+(?P<h>\d{1,2}):(?P<min>\d{2}))?'
)
RU_TEXTUAL_RE = re.compile(
    r'(?P<d>\d{1,2})\s+(?P<mon>[А-Яа-я]+)\s+(?P<y>\d{4})'
    r'(?:\s*г\.?|(?:\s*года)?)?(?:,\s*|\s+)?(?:(?P<h>\d{1,2}):(?P<min>\d{2}))?',
    re.IGNORECASE
)

def parse_russian_date(text: str, default_tz=timezone(timedelta(hours=3))) -> datetime | None:
    """Парсим '13 августа 2025, 16:49' или '13.08.2025 16:49'. Возвращаем UTC."""
    if not text:
        return None
    s = html.unescape(text.strip().lower())

    m = RU_NUMERIC_RE.search(s)
    if m:
        d, mth, y = int(m.group('d')), int(m.group('m')), int(m.group('y'))
        hh = int(m.group('h') or 0)
        mm = int(m.group('min') or 0)
        try:
            dt = datetime(y, mth, d, hh, mm, tzinfo=default_tz)
            return dt.astimezone(timezone.utc)
        except Exception:
            return None

    m = RU_TEXTUAL_RE.search(s)
    if m:
        d = int(m.group('d'))
        mon_name = m.group('mon')
        y = int(m.group('y'))
        mth = RU_MONTHS.get(mon_name, None)
        if not mth:
            return None
        hh = int(m.group('h') or 0)
        mm = int(m.group('min') or 0)
        try:
            dt = datetime(y, mth, d, hh, mm, tzinfo=default_tz)
            return dt.astimezone(timezone.utc)
        except Exception:
            return None
    return None

def extract_jsonld_datetime(soup: BeautifulSoup) -> datetime | None:
    """Читаем datePublished из JSON-LD (NewsArticle)."""
    for tag in soup.find_all("script", attrs={"type": "application/ld+json"}):
        try:
            data = json.loads(tag.string or tag.text or "")
        except Exception:
            continue
        objs = data if isinstance(data, list) else [data]
        for obj in objs:
            if not isinstance(obj, dict):
                continue
            dp = obj.get("datePublished") or obj.get("dateCreated")
            if dp:
                dt = parse_iso_or_rfc2822(dp)
                if dt:
                    return dt
    return None

def extract_datetime_generic(html_text: str, soup: BeautifulSoup) -> datetime | None:
    # 1) JSON-LD
    dt = extract_jsonld_datetime(soup)
    if dt:
        return dt
    # 2) <time datetime="..."> или <meta itemprop="datePublished" content="..."> или og:published_time
    t = soup.find('time', attrs={'datetime': True}) or soup.find('time', attrs={'content': True})
    if t:
        dt = parse_iso_or_rfc2822(t.get('datetime') or t.get('content'))
        if dt:
            return dt
    meta = soup.find('meta', attrs={'itemprop': 'datePublished'}) or \
           soup.find('meta', attrs={'property': 'article:published_time'})
    if meta and meta.get('content'):
        dt = parse_iso_or_rfc2822(meta['content'])
        if dt:
            return dt
    # 3) ISO в сыром HTML
    m = ISO_REGEX.search(html_text)
    if m:
        dt = parse_iso_or_rfc2822(m.group(0))
        if dt:
            return dt
    # 4) Русский текст (берём весь текст страницы)
    flat = ' '.join(soup.stripped_strings)
    return parse_russian_date(flat)

def extract_title_generic(soup: BeautifulSoup) -> str:
    h1 = soup.find('h1')
    if h1 and h1.get_text(strip=True):
        return h1.get_text(strip=True)
    og = soup.find('meta', attrs={'property': 'og:title'})
    if og and og.get('content'):
        return og['content'].strip()
    if soup.title and soup.title.text:
        return soup.title.text.strip()
    return ""

# ================= TASS =================
# Ловим ссылки статей раздела /ekonomika/<id>...
TASS_LINK_RE = re.compile(
    r'href=["\'](?:https?://(?:www\.)?tass\.ru)?(/ekonomika/\d+[^\s"\']*)["\']',
    re.IGNORECASE
)

def fetch_tass():
    out = []
    list_resp = _req(TASS_LIST_URL)
    if not list_resp:
        print("[TASS][ERROR] не удалось загрузить ленту")
        return out

    html_list = list_resp.text
    links = []
    seen = set()
    for m in TASS_LINK_RE.finditer(html_list):
        rel = m.group(1)
        if not rel:
            continue
        url = urllib.parse.urljoin(TASS_LIST_URL, rel)
        if url not in seen:
            seen.add(url)
            links.append(url)
        if len(links) >= MAX_LINKS_TASS:
            break

    if DEBUG:
        print(f"[TASS] кандидатов: {len(links)}")

    no_title = no_kw = no_dt = old_dt = 0
    for url in links:
        r = _req(url, timeout=20)
        if not r:
            continue
        soup = BeautifulSoup(r.text, "html.parser")
        title = extract_title_generic(soup)
        if not title:
            no_title += 1
            continue
        if not title_matches(title):
            no_kw += 1
            continue
        dt = extract_datetime_generic(r.text, soup)
        if not dt:
            no_dt += 1
            continue
        if not within_days(dt, DAYS_BACK):
            old_dt += 1
            continue
        out.append({'source': 'TASS', 'title': title, 'link': url, 'date': dt})
        if len(out) >= MAX_NEWS_PER_SITE:
            break

    if DEBUG:
        print(f"[TASS] прошло фильтр: {len(out)} | без заголовка: {no_title}, без KEYWORDS: {no_kw}, без даты: {no_dt}, старше {DAYS_BACK} дн.: {old_dt}")
    return out

# ================= Interfax =================
# Ссылки вида /business/123456
INTERFAX_LINK_RE = re.compile(
    r'href=["\'](?:https?://(?:www\.)?interfax\.ru)?(/business/\d+[^\s"\']*)["\']',
    re.IGNORECASE
)

def fetch_interfax():
    out = []
    list_resp = _req(INTERFAX_LIST_URL)
    if not list_resp:
        print("[Interfax][ERROR] не удалось загрузить ленту")
        return out

    html_list = list_resp.text
    links = []
    seen = set()
    for m in INTERFAX_LINK_RE.finditer(html_list):
        rel = m.group(1)
        if not rel:
            continue
        url = urllib.parse.urljoin(INTERFAX_LIST_URL, rel)
        if url not in seen:
            seen.add(url)
            links.append(url)
        if len(links) >= MAX_LINKS_INTERFAX:
            break

    if DEBUG:
        print(f"[Interfax] кандидатов: {len(links)}")

    no_title = no_kw = no_dt = old_dt = 0
    for url in links:
        r = _req(url, timeout=20)
        if not r:
            continue
        soup = BeautifulSoup(r.text, "html.parser")
        title = extract_title_generic(soup)
        if not title:
            no_title += 1
            continue
        if not title_matches(title):
            no_kw += 1
            continue
        dt = extract_datetime_generic(r.text, soup)
        if not dt:
            no_dt += 1
            continue
        if not within_days(dt, DAYS_BACK):
            old_dt += 1
            continue
        out.append({'source': 'Interfax', 'title': title, 'link': url, 'date': dt})
        if len(out) >= MAX_NEWS_PER_SITE:
            break

    if DEBUG:
        print(f"[Interfax] прошло фильтр: {len(out)} | без заголовка: {no_title}, без KEYWORDS: {no_kw}, без даты: {no_dt}, старше {DAYS_BACK} дн.: {old_dt}")
    return out

# ================= ВЫВОД =================
def print_articles(items):
    if not items:
        print("❌ Нет подходящих новостей.")
        return
    items.sort(key=lambda x: x['date'] or datetime.min.replace(tzinfo=timezone.utc), reverse=True)
    print(f"✅ Найдено {len(items)} новостей за {DAYS_BACK} дн.\n")

    # Jupyter: кликабельные ссылки
    try:
        from IPython.display import display, Markdown
        for i, a in enumerate(items, 1):
            # выводим по Asia/Bishkek (UTC+6), чтобы было в твоей TZ
            dt_local = a['date'].astimezone(timezone(timedelta(hours=6)))
            display(Markdown(
                f"**[{i}] [{a['source']}] {a['title']}**  \n"
                f"🗓 {dt_local.strftime('%Y-%m-%d %H:%M')}  \n"
                f"🔗 [{a['link']}]({a['link']})\n"
            ))
    except Exception:
        for i, a in enumerate(items, 1):
            dt_local = a['date'].astimezone(timezone(timedelta(hours=6)))
            print(f"[{i}] [{a['source']}] {a['title']}\n"
                  f"🗓 {dt_local.strftime('%Y-%m-%d %H:%M')}\n"
                  f"🔗 {a['link']}\n")

def main():
    tass_items = fetch_tass()
    interfax_items = fetch_interfax()
    all_items = tass_items + interfax_items
    print_articles(all_items)

if __name__ == "__main__":
    main()


✅ Найдено 5 новостей за 2 дн.



**[1] [TASS] Yonhap: Южная Корея и Вьетнам подписали соглашение о поставках самоходок K9**  
🗓 2025-08-14 07:58  
🔗 [https://tass.ru/ekonomika/24776477](https://tass.ru/ekonomika/24776477)


**[2] [TASS] Эксперт Богомолов: саммит РФ - США поможет развитию кроссполярных маршрутов**  
🗓 2025-08-14 07:47  
🔗 [https://tass.ru/ekonomika/24776451](https://tass.ru/ekonomika/24776451)


**[3] [TASS] Аксаков: с 2026 года россиянам запретят брать более одного займа в МФО**  
🗓 2025-08-14 07:34  
🔗 [https://tass.ru/ekonomika/24776439](https://tass.ru/ekonomika/24776439)


**[4] [TASS] В России отменили плановые проверки аудиторских организаций среднего риска**  
🗓 2025-08-14 01:18  
🔗 [https://tass.ru/ekonomika/24775349](https://tass.ru/ekonomika/24775349)


**[5] [TASS] Бразилия для ответа на тарифы США организует переговоры членов БРИКС**  
🗓 2025-08-14 01:03  
🔗 [https://tass.ru/ekonomika/24775379](https://tass.ru/ekonomika/24775379)


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re

# === НАСТРОЙКИ ===
KEYWORDS = ['трамп', 'канад', 'германи', 'сша', 'австрали', 'япони', 'великобритани', 'китай', 'росси']
DAYS_BACK = 2
URL = 'https://www.rbc.ru/economics/'

# === ЗАПУСК SELENIUM ===
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(options=chrome_options)

print("[INFO] Загружаем страницу...")
driver.get(URL)
time.sleep(5)  # Лучше заменить на явные ожидания для продакшена

html = driver.page_source
driver.quit()

# === ПАРСИНГ И ОДНОВРЕМЕННАЯ ФИЛЬТРАЦИЯ ===
soup = BeautifulSoup(html, 'html.parser')
items = soup.find_all('div', class_='item')
print(f"[INFO] Найдено карточек на странице: {len(items)}")

now = datetime.now()
filtered_articles = []

for item in items:
    title_tag = item.find('span', class_='item__title')
    link_tag = item.find('a', class_='item__link')

    if not title_tag or not link_tag:
        continue

    title = ''.join(title_tag.stripped_strings)
    link = link_tag['href']

    # Извлекаем дату из URL
    match = re.search(r'/(\d{2})/(\d{2})/(\d{4})/', link)
    if not match:
        continue
    day, month, year = map(int, match.groups())
    pub_date = datetime(year, month, day)

    # Фильтрация по дате — пропускаем новости старше DAYS_BACK
    if (now - pub_date).days > DAYS_BACK:
        continue

    # Фильтрация по ключевым словам — пропускаем, если не содержат ни одного ключевого слова
    title_lower = title.lower()
    if not any(kw in title_lower for kw in KEYWORDS):
        continue

    filtered_articles.append({
        'title': title,
        'link': link,
        'date': pub_date
    })

# === ВЫВОД РЕЗУЛЬТАТОВ ===
if not filtered_articles:
    print("\n❌ Нет подходящих новостей.")
else:
    print(f"\n✅ Найдено {len(filtered_articles)} новостей по фильтру:\n")
    for i, a in enumerate(filtered_articles, 1):
        print(f"[{i}] [{a['date'].strftime('%Y-%m-%d')}] {a['title']}\n     🔗 {a['link']}\n")


[INFO] Загружаем страницу...
[INFO] Найдено карточек на странице: 20

✅ Найдено 4 новостей по фильтру:

[1] [2025-08-13] Росстат сообщил о замедлении роста российской экономики
     🔗 https://www.rbc.ru/economics/13/08/2025/689cbb6f9a7947396a1ea3a7

[2] [2025-08-13] В Индии рассказали о защите от рисков для рынка нефти из-за пошлин Трампа
     🔗 https://www.rbc.ru/economics/13/08/2025/689bbb429a794773357cb68a

[3] [2025-08-11] Трамп поставил Китаю новый дедлайн по пошлинам
     🔗 https://www.rbc.ru/economics/11/08/2025/689a36c79a7947ba41acd89e

[4] [2025-08-11] Трамп отменил «удар» по золоту Швейцарии
     🔗 https://www.rbc.ru/economics/11/08/2025/689a2c119a7947d7cfde870c



In [ ]:
import re
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from datetime import datetime, timedelta
from IPython.display import display, Markdown

KEYWORDS = ['трамп', 'trump', 'дональд']
DAYS_BACK = 2
URL = 'https://iz.ru/news'
MAX_NEWS = 50
BASE_URL = 'https://iz.ru'

def extract_json_from_html(html):
    start_pattern = r'window\.recommendationBlockList\s*=\s*{'
    start_match = re.search(start_pattern, html)
    if not start_match:
        return None

    start_index = start_match.end() - 1  # позиция первой {

    braces = 0
    end_index = start_index
    for i, ch in enumerate(html[start_index:], start=start_index):
        if ch == '{':
            braces += 1
        elif ch == '}':
            braces -= 1
            if braces == 0:
                end_index = i
                break

    json_text = html[start_index:end_index+1]
    return json_text

def parse_articles(json_data):
    articles = []
    for key in ['even', 'odd']:
        for item in json_data.get(key, []):
            path = item.get('path', '')
            if path.startswith('http://') or path.startswith('https://'):
                full_link = path
            else:
                full_link = BASE_URL + path if path.startswith('/') else BASE_URL + '/' + path
            articles.append({
                'title': item.get('title', ''),
                'link': full_link,
                'date': None,
                'reference': item.get('reference', '')
            })
    return articles

def main():
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(URL)

    html = driver.page_source
    driver.quit()

    json_text = extract_json_from_html(html)
    if not json_text:
        print("❌ Не удалось найти данные новостей в HTML")
        return

    data = json.loads(json_text)
    articles = parse_articles(data)

    filtered = []
    now = datetime.now()
    cutoff = now - timedelta(days=DAYS_BACK)

    for a in articles:
        title_lower = a['title'].lower()
        if not any(kw in title_lower for kw in KEYWORDS):
            continue
        filtered.append(a)
        if len(filtered) >= MAX_NEWS:
            break

    if not filtered:
        print("❌ Нет подходящих новостей.")
        return

    print(f"✅ Найдено {len(filtered)} новостей по фильтру:\n")
    for i, a in enumerate(filtered, 1):
        # Вывод кликаемой ссылки через Markdown
        display(Markdown(f"**[{i}] {a['title']}**  \n🔗 [{a['link']}]({a['link']})\n"))

if __name__ == '__main__':
    main()


❌ Не удалось найти данные новостей в HTML


In [ ]:
import re
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import time

KEYWORDS = ['трамп', 'trump', 'дональд', 'канад', 'германи', 'сша', 'австрали', 'япони', 'великобритани', 'китай', 'росси']
DAYS_BACK = 2
MAX_NEWS = 50

def parse_rbc(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_='item')
    now = datetime.now()
    filtered_articles = []

    for item in items:
        title_tag = item.find('span', class_='item__title')
        link_tag = item.find('a', class_='item__link')

        if not title_tag or not link_tag:
            continue

        title = ''.join(title_tag.stripped_strings)
        link = link_tag['href']

        match = re.search(r'/(\d{2})/(\d{2})/(\d{4})/', link)
        if not match:
            continue
        day, month, year = map(int, match.groups())
        pub_date = datetime(year, month, day)

        if (now - pub_date).days > DAYS_BACK:
            continue

        title_lower = title.lower()
        if not any(kw in title_lower for kw in KEYWORDS):
            continue

        filtered_articles.append({
            'title': title,
            'link': link,
            'date': pub_date
        })

    return filtered_articles

def extract_json_from_html(html):
    start_pattern = r'window\.recommendationBlockList\s*=\s*{'
    start_match = re.search(start_pattern, html)
    if not start_match:
        return None

    start_index = start_match.end() - 1
    braces = 0
    end_index = start_index
    for i, ch in enumerate(html[start_index:], start=start_index):
        if ch == '{':
            braces += 1
        elif ch == '}':
            braces -= 1
            if braces == 0:
                end_index = i
                break

    return html[start_index:end_index+1]

def parse_iz(json_data):
    BASE_URL = 'https://iz.ru'
    articles = []
    for key in ['even', 'odd']:
        for item in json_data.get(key, []):
            path = item.get('path', '')
            if path.startswith('http://') or path.startswith('https://'):
                full_link = path
            else:
                full_link = BASE_URL + path if path.startswith('/') else BASE_URL + '/' + path
            articles.append({
                'title': item.get('title', ''),
                'link': full_link,
                'date': None,
            })
    return articles

def fetch_with_selenium(url):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(5)  # можно заменить на явные ожидания
    html = driver.page_source
    driver.quit()
    return html

def main():
    # --- RBC ---
    print("[INFO] Загружаем RBC...")
    rbc_html = fetch_with_selenium('https://www.rbc.ru/economics/')
    rbc_articles = parse_rbc(rbc_html)

    # --- Iz.ru ---
    print("[INFO] Загружаем Iz.ru...")
    iz_html = fetch_with_selenium('https://iz.ru/news')
    json_text = extract_json_from_html(iz_html)
    if not json_text:
        print("❌ Не удалось найти новости Iz.ru")
        iz_articles = []
    else:
        data = json.loads(json_text)
        iz_articles = parse_iz(data)

    # Объединяем статьи
    all_articles = rbc_articles + iz_articles

    # Фильтруем по ключевым словам и дате для Iz.ru (даты нет, пропускаем фильтр по времени)
    now = datetime.now()
    cutoff = now - timedelta(days=DAYS_BACK)

    filtered = []
    for a in all_articles:
        title_lower = a['title'].lower()
        if not any(kw in title_lower for kw in KEYWORDS):
            continue
        # Для RBC проверка даты уже сделана
        # Для Iz.ru даты нет, считаем допустимыми все
        filtered.append(a)
        if len(filtered) >= MAX_NEWS:
            break

    if not filtered:
        print("❌ Нет подходящих новостей.")
        return

    print(f"✅ Найдено {len(filtered)} новостей по фильтру:\n")
    for i, a in enumerate(filtered, 1):
        # В Jupyter делаем кликабельный вывод
        display(Markdown(f"**[{i}] {a['title']}**  \n🔗 [{a['link']}]({a['link']})\n"))

if __name__ == '__main__':
    main()


[INFO] Загружаем RBC...
[INFO] Загружаем Iz.ru...
❌ Не удалось найти новости Iz.ru
✅ Найдено 4 новостей по фильтру:



**[1] Росстат сообщил о замедлении роста российской экономики**  
🔗 [https://www.rbc.ru/economics/13/08/2025/689cbb6f9a7947396a1ea3a7](https://www.rbc.ru/economics/13/08/2025/689cbb6f9a7947396a1ea3a7)


**[2] В Индии рассказали о защите от рисков для рынка нефти из-за пошлин Трампа**  
🔗 [https://www.rbc.ru/economics/13/08/2025/689bbb429a794773357cb68a](https://www.rbc.ru/economics/13/08/2025/689bbb429a794773357cb68a)


**[3] Трамп поставил Китаю новый дедлайн по пошлинам**  
🔗 [https://www.rbc.ru/economics/11/08/2025/689a36c79a7947ba41acd89e](https://www.rbc.ru/economics/11/08/2025/689a36c79a7947ba41acd89e)


**[4] Трамп отменил «удар» по золоту Швейцарии**  
🔗 [https://www.rbc.ru/economics/11/08/2025/689a2c119a7947d7cfde870c](https://www.rbc.ru/economics/11/08/2025/689a2c119a7947d7cfde870c)


In [ ]:
import re
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from IPython.display import display, Markdown  # для кликабельных ссылок в Jupyter, если не используешь — заменить на print

# === НАСТРОЙКИ ===
KEYWORDS = ['трамп', 'trump', 'дональд', 'канад', 'германи', 'сша', 'австрали', 'япони', 'великобритани', 'китай', 'росси']
DAYS_BACK = 2
MAX_NEWS_IZ = 50

# --- URLs ---
RBC_URL = 'https://www.rbc.ru/economics/'
IZ_URL = 'https://iz.ru/news'
IZ_BASE_URL = 'https://iz.ru'

def fetch_rbc_articles(driver, now):
    driver.get(RBC_URL)
    driver.implicitly_wait(5)  # лучше использовать явные ожидания, но для простоты
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_='item')
    articles = []

    for item in items:
        title_tag = item.find('span', class_='item__title')
        link_tag = item.find('a', class_='item__link')

        if not title_tag or not link_tag:
            continue

        title = ''.join(title_tag.stripped_strings)
        link = link_tag['href']

        # Извлекаем дату из URL
        match = re.search(r'/(\d{2})/(\d{2})/(\d{4})/', link)
        if not match:
            continue
        day, month, year = map(int, match.groups())
        pub_date = datetime(year, month, day)

        # Фильтр по дате
        if (now - pub_date).days > DAYS_BACK:
            continue

        # Фильтр по ключевым словам
        if not any(kw in title.lower() for kw in KEYWORDS):
            continue

        articles.append({
            'title': title,
            'link': link,
            'date': pub_date,
        })

    return articles

def extract_json_from_html(html):
    start_pattern = r'window\.recommendationBlockList\s*=\s*{'
    start_match = re.search(start_pattern, html)
    if not start_match:
        return None

    start_index = start_match.end() - 1
    braces = 0
    end_index = start_index
    for i, ch in enumerate(html[start_index:], start=start_index):
        if ch == '{':
            braces += 1
        elif ch == '}':
            braces -= 1
            if braces == 0:
                end_index = i
                break
    json_text = html[start_index:end_index+1]
    return json_text

def parse_iz(json_data):
    articles_dict = {}
    for key in ['even', 'odd']:
        for item in json_data.get(key, []):
            article_id = item.get('id') or item.get('reference') or item.get('path')
            path = item.get('path', '').strip()
            if not path:
                path = item.get('reference', '').strip()

            # Проверка валидности ссылки
            if not path or path in ['/', IZ_BASE_URL, IZ_BASE_URL + '/']:
                print(f"[WARN] Пропущена новость без валидной ссылки: Заголовок: \"{item.get('title', '')}\"")
                continue

            if path.startswith('http://') or path.startswith('https://'):
                full_link = path
            else:
                full_link = IZ_BASE_URL + path if path.startswith('/') else IZ_BASE_URL + '/' + path

            unique_key = article_id if article_id else full_link

            if unique_key in articles_dict:
                continue

            articles_dict[unique_key] = {
                'title': item.get('title', ''),
                'link': full_link,
                'date': None,  # в данных iz.ru даты нет, можно потом пытаться парсить с сайта, но это сложнее
            }

    return list(articles_dict.values())

def fetch_iz_articles(driver, now):
    driver.get(IZ_URL)
    driver.implicitly_wait(5)
    html = driver.page_source

    json_text = extract_json_from_html(html)
    if not json_text:
        print("❌ Не удалось найти данные новостей iz.ru в HTML")
        return []

    data = json.loads(json_text)
    articles = parse_iz(data)

    filtered = []
    for a in articles:
        if not any(kw in a['title'].lower() for kw in KEYWORDS):
            continue
        filtered.append(a)
        if len(filtered) >= MAX_NEWS_IZ:
            break

    return filtered

def print_articles(articles):
    if not articles:
        print("❌ Нет подходящих новостей.")
        return
    print(f"✅ Найдено {len(articles)} новостей по фильтру:\n")
    for i, a in enumerate(articles, 1):
        # Кликабельные ссылки, если запускаешь в Jupyter/IPython
        try:
            display(Markdown(f"**[{i}] {a['title']}**  \n🔗 [{a['link']}]({a['link']})\n"))
        except NameError:
            # Если display или Markdown не доступны — простой вывод
            print(f"[{i}] {a['title']}\n🔗 {a['link']}\n")

def main():
    now = datetime.now()

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=chrome_options)

    # Парсим RBC
    rbc_articles = fetch_rbc_articles(driver, now)

    # Парсим Iz.ru
    iz_articles = fetch_iz_articles(driver, now)

    driver.quit()

    # Объединяем результаты
    all_articles = rbc_articles + iz_articles

    # Сортируем по дате — у iz.ru даты нет, поэтому они идут после RBC
    all_articles.sort(key=lambda x: x['date'] if x['date'] else datetime.min, reverse=True)

    print_articles(all_articles)

if __name__ == '__main__':
    main()


❌ Не удалось найти данные новостей iz.ru в HTML
✅ Найдено 4 новостей по фильтру:



**[1] Росстат сообщил о замедлении роста российской экономики**  
🔗 [https://www.rbc.ru/economics/13/08/2025/689cbb6f9a7947396a1ea3a7](https://www.rbc.ru/economics/13/08/2025/689cbb6f9a7947396a1ea3a7)


**[2] В Индии рассказали о защите от рисков для рынка нефти из-за пошлин Трампа**  
🔗 [https://www.rbc.ru/economics/13/08/2025/689bbb429a794773357cb68a](https://www.rbc.ru/economics/13/08/2025/689bbb429a794773357cb68a)


**[3] Трамп поставил Китаю новый дедлайн по пошлинам**  
🔗 [https://www.rbc.ru/economics/11/08/2025/689a36c79a7947ba41acd89e](https://www.rbc.ru/economics/11/08/2025/689a36c79a7947ba41acd89e)


**[4] Трамп отменил «удар» по золоту Швейцарии**  
🔗 [https://www.rbc.ru/economics/11/08/2025/689a2c119a7947d7cfde870c](https://www.rbc.ru/economics/11/08/2025/689a2c119a7947d7cfde870c)


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import re

KEYWORDS = ['экономика', 'рынок', 'инфляция', 'курс', 'банк', 'цена', 'снижение', 'рост', 'трамп', 'trump', 'дональд']
DAYS_BACK = 2
URL = 'https://tass.ru/ekonomika'
TARGET_NEWS_COUNT = 100
SCROLL_PAUSE_SEC = 2  # пауза после скролла для загрузки

def scroll_to_load(driver, target_count):
    last_height = driver.execute_script("return document.body.scrollHeight")
    news_count = 0
    tries = 0
    max_tries = 20  # чтобы не зациклиться

    while news_count < target_count and tries < max_tries:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_SEC)

        # Проверяем сколько карточек сейчас на странице
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles = soup.find_all('article', class_=re.compile(r'list-item|card-mini'))
        news_count = len(articles)
        if news_count >= target_count:
            break

        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            tries += 1
        else:
            tries = 0
            last_height = new_height

def parse_tass_news():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(options=chrome_options)
    driver.set_page_load_timeout(180)

    print("[INFO] Загружаем страницу TASS...")
    try:
        driver.get(URL)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, 'article'))
        )
        scroll_to_load(driver, TARGET_NEWS_COUNT)
    except Exception as e:
        print(f"[ERROR] Ошибка при загрузке страницы: {e}")
        driver.quit()
        return

    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.find_all('article', class_=re.compile(r'list-item|card-mini'))
    print(f"[INFO] Всего найдено новостей после скролла: {len(articles)}")

    now = datetime.now()
    cutoff_date = now - timedelta(days=DAYS_BACK)

    filtered_articles = []

    for article in articles:
        title_tag = article.find('a', class_='list-item__title') or article.find('a', class_='card-mini__title')
        if not title_tag:
            continue
        title = title_tag.get_text(strip=True)
        link = title_tag['href']

        time_tag = article.find('time')
        if time_tag and time_tag.has_attr('datetime'):
            pub_date_str = time_tag['datetime']
            try:
                pub_date = datetime.fromisoformat(pub_date_str.replace('Z', '+00:00')).replace(tzinfo=None)
            except Exception:
                continue
        else:
            continue

        if pub_date < cutoff_date:
            continue

        title_lower = title.lower()
        if not any(kw in title_lower for kw in KEYWORDS):
            continue

        if link.startswith('/'):
            link = 'https://tass.ru' + link

        filtered_articles.append({
            'title': title,
            'link': link,
            'date': pub_date
        })

        if len(filtered_articles) >= TARGET_NEWS_COUNT:
            break

    if not filtered_articles:
        print("❌ Нет подходящих новостей.")
        return

    print(f"\n✅ Найдено {len(filtered_articles)} новостей по фильтру за последние {DAYS_BACK} дня:\n")
    for i, a in enumerate(filtered_articles, 1):
        print(f"[{i}] [{a['date'].strftime('%Y-%m-%d %H:%M')}] {a['title']}\n     🔗 {a['link']}\n")

if __name__ == "__main__":
    parse_tass_news()


[INFO] Загружаем страницу TASS...
[ERROR] Ошибка при загрузке страницы: Message: 
Stacktrace:
#0 0x5a881382601a <unknown>
#1 0x5a88132c5a70 <unknown>
#2 0x5a8813317907 <unknown>
#3 0x5a8813317b01 <unknown>
#4 0x5a8813365d54 <unknown>
#5 0x5a881333d40d <unknown>
#6 0x5a881336314f <unknown>
#7 0x5a881333d1b3 <unknown>
#8 0x5a881330959b <unknown>
#9 0x5a881330a971 <unknown>
#10 0x5a88137eb1eb <unknown>
#11 0x5a88137eef39 <unknown>
#12 0x5a88137d22c9 <unknown>
#13 0x5a88137efae8 <unknown>
#14 0x5a88137b6baf <unknown>
#15 0x5a88138130a8 <unknown>
#16 0x5a8813813286 <unknown>
#17 0x5a8813824ff6 <unknown>
#18 0x7d6c49916ac3 <unknown>



In [ ]:
import re
import json
from datetime import datetime, timedelta
from urllib.parse import urlparse

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

# === НАСТРОЙКИ ===
KEYWORDS = ['трамп', 'trump', 'дональд', 'канад', 'германи', 'сша', 'австрали', 'япони', 'великобритани', 'китай', 'росси']
DAYS_BACK = 2

# лимиты (чтобы не перебирать лишнего)
MAX_NEWS_RBC = 50
MAX_NEWS_IZ  = 50

# --- URLs ---
RBC_URL = 'https://www.rbc.ru/economics/'
IZ_URL  = 'https://iz.ru/news'
IZ_BASE_URL = 'https://iz.ru'

# Markdown-вывод (кликабельно в Jupyter), иначе fallback на print
def emit_item(i, title, link, dt=None):
    try:
        from IPython.display import display, Markdown
        date_str = f"[{dt.strftime('%Y-%m-%d')}]" if dt else ""
        display(Markdown(f"**[{i}] {date_str} {title}**  \n🔗 [{link}]({link})\n"))
    except Exception:
        date_str = f"[{dt.strftime('%Y-%m-%d')}]" if dt else ""
        print(f"[{i}] {date_str} {title}\n🔗 {link}\n")

def build_driver() -> webdriver.Chrome:
    chrome_options = Options()
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-gpu")
    # чуть ускоряем загрузку
    chrome_prefs = {
        "profile.managed_default_content_settings.images": 2,  # без картинок
        "profile.default_content_setting_values.cookies": 2,    # без кук (опционально)
    }
    chrome_options.add_experimental_option("prefs", chrome_prefs)
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                                "AppleWebKit/537.36 (KHTML, like Gecko) "
                                "Chrome/125.0 Safari/537.36")

    # если нужен явный путь к chromedriver:
    # service = Service("/path/to/chromedriver")
    # return webdriver.Chrome(service=service, options=chrome_options)
    return webdriver.Chrome(options=chrome_options)

# ---------- RBC ----------
def fetch_rbc_articles(driver, now: datetime):
    driver.set_page_load_timeout(60)
    driver.get(RBC_URL)
    # ждём появления карточек
    WebDriverWait(driver, 20).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.item"))
    )
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    items = soup.find_all('div', class_='item')
    articles = []
    seen = set()  # дедуп по ссылке

    date_rx = re.compile(r'/(\d{2})/(\d{2})/(\d{4})/')
    cutoff = now - timedelta(days=DAYS_BACK)

    for item in items:
        if len(articles) >= MAX_NEWS_RBC:
            break

        title_tag = item.find('span', class_='item__title')
        link_tag  = item.find('a',   class_='item__link')
        if not title_tag or not link_tag:
            continue

        title = ''.join(title_tag.stripped_strings)
        link  = link_tag.get('href', '').strip()
        if not link or link in seen:
            continue

        m = date_rx.search(link)
        if not m:
            continue
        day, month, year = map(int, m.groups())
        pub_date = datetime(year, month, day)

        if pub_date < cutoff:
            continue
        if not any(kw in title.lower() for kw in KEYWORDS):
            continue

        seen.add(link)
        articles.append({
            'title': title,
            'link':  link,
            'date':  pub_date,
        })

    return articles

# ---------- IZ ----------
def extract_reco_json(html: str) -> str | None:
    """
    Аккуратно достаём тело JSON из window.recommendationBlockList = {...};
    балансируя фигурные скобки.
    """
    start_match = re.search(r'window\.recommendationBlockList\s*=\s*{', html)
    if not start_match:
        return None
    start = start_match.end() - 1
    braces = 0
    end = start
    for i, ch in enumerate(html[start:], start=start):
        if ch == '{':
            braces += 1
        elif ch == '}':
            braces -= 1
            if braces == 0:
                end = i
                break
    return html[start:end+1]

def parse_iz_json(data: dict, allow_only_iz_domain: bool = True):
    """
    Собираем статьи из блоков even/odd.
    - Убираем дубли (по id/reference/path или ссылке)
    - Опционально фильтруем внешние домены (оставляем только iz.ru)
    """
    articles_dict = {}
    for key in ('even', 'odd'):
        for item in data.get(key, []) or []:
            article_id = item.get('id') or item.get('reference') or item.get('path')
            raw_path = (item.get('path') or item.get('reference') or '').strip()
            if not raw_path:
                print(f"[WARN] Пропущена новость без ссылки: «{item.get('title', '')}»")
                continue

            if raw_path.startswith('http://') or raw_path.startswith('https://'):
                full_link = raw_path
            else:
                full_link = IZ_BASE_URL + raw_path if raw_path.startswith('/') else IZ_BASE_URL + '/' + raw_path

            if allow_only_iz_domain:
                dom = urlparse(full_link).netloc
                if 'iz.ru' not in dom:
                    # пропускаем внешние репосты, чтобы не тащить sport-express/regnum и т.п.
                    continue

            unique_key = str(article_id) if article_id else full_link
            if unique_key in articles_dict:
                continue

            articles_dict[unique_key] = {
                'title': item.get('title', '').strip(),
                'link':  full_link,
                'date':  None,  # даты в этом JSON нет
            }
    return list(articles_dict.values())

def fetch_iz_articles(driver):
    driver.set_page_load_timeout(60)
    driver.get(IZ_URL)
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "script"))
    )
    html = driver.page_source
    json_text = extract_reco_json(html)
    if not json_text:
        print("❌ Iz: не найден window.recommendationBlockList")
        return []

    data = json.loads(json_text)
    articles = parse_iz_json(data, allow_only_iz_domain=True)

    # предварительная фильтрация по ключевым словам
    out = []
    for a in articles:
        if any(kw in a['title'].lower() for kw in KEYWORDS):
            out.append(a)
            if len(out) >= MAX_NEWS_IZ:
                break
    return out

# ---------- MAIN ----------
def main():
    now = datetime.now()
    driver = build_driver()

    try:
        rbc_articles = fetch_rbc_articles(driver, now)
        iz_articles  = fetch_iz_articles(driver)
    finally:
        driver.quit()

    # объединяем и сортируем (у IZ даты нет — они пойдут после RBC)
    all_articles = rbc_articles + iz_articles
    all_articles.sort(key=lambda x: x['date'] if x['date'] else datetime.min, reverse=True)

    if not all_articles:
        print("❌ Нет подходящих новостей.")
        return

    print(f"✅ Найдено {len(all_articles)} новостей по фильтру (RBC + Iz):\n")
    for i, a in enumerate(all_articles, 1):
        emit_item(i, a['title'], a['link'], a.get('date'))

if __name__ == '__main__':
    main()


❌ Iz: не найден window.recommendationBlockList
✅ Найдено 2 новостей по фильтру (RBC + Iz):



**[1] [2025-08-13] Росстат сообщил о замедлении роста российской экономики**  
🔗 [https://www.rbc.ru/economics/13/08/2025/689cbb6f9a7947396a1ea3a7](https://www.rbc.ru/economics/13/08/2025/689cbb6f9a7947396a1ea3a7)


**[2] [2025-08-13] В Индии рассказали о защите от рисков для рынка нефти из-за пошлин Трампа**  
🔗 [https://www.rbc.ru/economics/13/08/2025/689bbb429a794773357cb68a](https://www.rbc.ru/economics/13/08/2025/689bbb429a794773357cb68a)


In [ ]:
import re
import sys
import time
import html
import json
import urllib.parse
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta, timezone

import requests
from bs4 import BeautifulSoup

# ========= НАСТРОЙКИ =========
KEYWORDS = ['трамп', 'trump', 'дональд', 'сша', 'германи', 'китай', 'росси', 'эконом', 'инфляц', 'ставк']
DAYS_BACK = 2
MAX_NEWS_PER_SITE = 50
MAX_LINKS_FINMARKET = 100   # ограничим число карточек, которые открываем у Finmarket

# DW RSS — максимально быстро и стабильно
DW_RSS_URL = "https://rss.dw.com/rdf/rss-ru-all"

# Finmarket лента
FINMARKET_LIST_URL = "https://www.finmarket.ru/news/"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/124.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "ru,en;q=0.9",
    "Connection": "keep-alive",
}

# Русские месяцы → номер
RU_MONTHS = {
    'января': 1, 'февраля': 2, 'марта': 3, 'апреля': 4, 'мая': 5, 'июня': 6,
    'июля': 7, 'августа': 8, 'сентября': 9, 'октября': 10, 'ноября': 11, 'декабря': 12
}

def now_utc():
    return datetime.now(timezone.utc)

def within_days(dt: datetime, days: int) -> bool:
    """Проверка, что dt в пределах последних days суток относительно текущего UTC."""
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return (now_utc() - dt) <= timedelta(days=days)

def title_matches(title: str) -> bool:
    tl = title.lower()
    return any(kw in tl for kw in KEYWORDS)

# ---------- Парсинг дат ----------

def parse_iso_or_rfc2822(s: str) -> datetime | None:
    """Пытаемся распарсить ISO8601 или RFC2822 (из RSS). Возвращаем tz-aware UTC."""
    s = s.strip()
    # ISO 8601
    try:
        # Приводим 'Z' к +00:00
        if s.endswith('Z'):
            dt = datetime.fromisoformat(s.replace('Z', '+00:00'))
        else:
            dt = datetime.fromisoformat(s)
        return dt.astimezone(timezone.utc)
    except Exception:
        pass
    # RFC 2822 (Wed, 07 Aug 2025 18:04:00 +0000)
    try:
        from email.utils import parsedate_to_datetime
        dt = parsedate_to_datetime(s)
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=timezone.utc)
        return dt.astimezone(timezone.utc)
    except Exception:
        return None

_RU_NUMERIC_RE = re.compile(r'(?P<d>\d{1,2})\.(?P<m>\d{1,2})\.(?P<y>\d{4})(?:\s+(?P<h>\d{1,2}):(?P<min>\d{2}))?')
_RU_TEXTUAL_RE = re.compile(
    r'(?P<d>\d{1,2})\s+(?P<mon>[А-Яа-я]+)\s+(?P<y>\d{4})'
    r'(?:\s*г\.?|(?:\s*года)?)?(?:\s+(?P<h>\d{1,2}):(?P<min>\d{2}))?',
    re.IGNORECASE
)

def parse_russian_date(text: str) -> datetime | None:
    """Парсим русские даты вида '8 августа 2025 19:04' или '08.08.2025 19:04'. Возвращаем UTC."""
    if not text:
        return None
    s = html.unescape(text.strip().lower())
    # 1) dd.mm.yyyy [hh:mm]
    m = _RU_NUMERIC_RE.search(s)
    if m:
        d, mth, y = int(m.group('d')), int(m.group('m')), int(m.group('y'))
        hh = int(m.group('h')) if m.group('h') else 0
        mm = int(m.group('min')) if m.group('min') else 0
        try:
            # считаем время московским (MSK, UTC+3) как sane default, затем в UTC
            dt = datetime(y, mth, d, hh, mm, tzinfo=timezone(timedelta(hours=3)))
            return dt.astimezone(timezone.utc)
        except Exception:
            return None

    # 2) 8 августа 2025 [19:04]
    m = _RU_TEXTUAL_RE.search(s)
    if m:
        d = int(m.group('d'))
        mon_name = m.group('mon')
        y = int(m.group('y'))
        mth = RU_MONTHS.get(mon_name, None)
        if not mth:
            return None
        hh = int(m.group('h')) if m.group('h') else 0
        mm = int(m.group('min')) if m.group('min') else 0
        try:
            dt = datetime(y, mth, d, hh, mm, tzinfo=timezone(timedelta(hours=3)))
            return dt.astimezone(timezone.utc)
        except Exception:
            return None
    return None

# ---------- DW: RSS ----------

def fetch_dw_articles() -> list[dict]:
    out = []
    try:
        resp = requests.get(DW_RSS_URL, headers=HEADERS, timeout=20)
        resp.raise_for_status()
        # RSS RDF: парсим через xml.etree
        root = ET.fromstring(resp.content)
        # Ищем items без заморочек с пространствами имён: перебором
        items = [el for el in root.iter() if el.tag.lower().endswith('item')]
        for it in items:
            title = None
            link = None
            date_text = None
            for child in it:
                tag = child.tag.lower()
                if tag.endswith('title'):
                    title = (child.text or '').strip()
                elif tag.endswith('link'):
                    link = (child.text or '').strip()
                elif tag.endswith('date') or tag.endswith('pubdate'):
                    date_text = (child.text or '').strip()
            if not title or not link:
                continue
            if not title_matches(title):
                continue
            pub_dt = parse_iso_or_rfc2822(date_text) if date_text else None
            # Если даты нет — возьмём как старую (выпадет фильтром)
            if not pub_dt or not within_days(pub_dt, DAYS_BACK):
                continue
            out.append({
                'source': 'DW',
                'title': title,
                'link': link,
                'date': pub_dt
            })
            if len(out) >= MAX_NEWS_PER_SITE:
                break
    except Exception as e:
        print(f"[DW][ERROR] {e}")
    return out

# ---------- Finmarket: HTML + карточки ----------

_FINMARKET_LINK_RE = re.compile(r'href=["\'](/news/\d{6,})["\']', re.IGNORECASE)

def _request(url: str, timeout: int = 20) -> requests.Response | None:
    try:
        r = requests.get(url, headers=HEADERS, timeout=timeout)
        # некоторые страницы отдают cp1251 — уважим явную кодировку/аппарентную
        enc = r.encoding or r.apparent_encoding or 'utf-8'
        r.encoding = enc
        if r.status_code == 200:
            return r
        return None
    except Exception:
        return None

def _extract_finmarket_article_date(html_text: str) -> datetime | None:
    """
    На странице новости Finmarket ищем дату в человекочитаемом виде.
    Примеры: '08 августа 2025 года 19:04', '08.08.2025 19:04'
    """
    # срежем лишнее и ищем по всему тексту
    txt = ' '.join(BeautifulSoup(html_text, 'html.parser').stripped_strings)
    # сначала numeric
    dt = parse_russian_date(txt)
    return dt

def _extract_finmarket_article_title(html_text: str) -> str:
    soup = BeautifulSoup(html_text, 'html.parser')
    # пробуем h1
    h1 = soup.find('h1')
    if h1 and h1.get_text(strip=True):
        return h1.get_text(strip=True)
    # fallback: title
    if soup.title and soup.title.text:
        return soup.title.text.strip()
    return ""

def fetch_finmarket_articles() -> list[dict]:
    out = []
    seen = set()
    resp = _request(FINMARKET_LIST_URL)
    if not resp:
        print("[Finmarket][ERROR] не удалось загрузить ленту.")
        return out

    html_text = resp.text
    # собираем кандидатов из ленты (сохраняем порядок появления)
    candidates = []
    for m in _FINMARKET_LINK_RE.finditer(html_text):
        path = m.group(1)
        if path not in seen:
            seen.add(path)
            candidates.append(urllib.parse.urljoin(FINMARKET_LIST_URL, path))
        if len(candidates) >= MAX_LINKS_FINMARKET:
            break

    for url in candidates:
        art_resp = _request(url, timeout=20)
        if not art_resp:
            continue
        art_html = art_resp.text
        title = _extract_finmarket_article_title(art_html)
        if not title or not title_matches(title):
            continue
        pub_dt = _extract_finmarket_article_date(art_html)
        if not pub_dt or not within_days(pub_dt, DAYS_BACK):
            continue
        out.append({
            'source': 'Finmarket',
            'title': title,
            'link': url,
            'date': pub_dt
        })
        if len(out) >= MAX_NEWS_PER_SITE:
            break
    return out

# ---------- Вывод ----------

def print_articles(articles: list[dict]):
    if not articles:
        print("❌ Нет подходящих новостей.")
        return

    # Сортируем по дате по убыванию
    articles.sort(key=lambda x: x['date'] or datetime.min.replace(tzinfo=timezone.utc), reverse=True)

    print(f"✅ Найдено {len(articles)} новостей (за {DAYS_BACK} дн.)\n")
    # Кликабельные ссылки в Jupyter
    try:
        from IPython.display import display, Markdown
        for i, a in enumerate(articles, 1):
            dt_local = a['date'].astimezone(timezone(timedelta(hours=6)))  # Asia/Bishkek UTC+6
            display(Markdown(
                f"**[{i}] [{a['source']}] {a['title']}**  \n"
                f"🗓 {dt_local.strftime('%Y-%m-%d %H:%M')}  \n"
                f"🔗 [{a['link']}]({a['link']})\n"
            ))
    except Exception:
        for i, a in enumerate(articles, 1):
            dt_local = a['date'].astimezone(timezone(timedelta(hours=6)))
            print(f"[{i}] [{a['source']}] {a['title']}\n"
                  f"🗓 {dt_local.strftime('%Y-%m-%d %H:%M')}\n"
                  f"🔗 {a['link']}\n")

def main():
    # 1) DW через RSS (очень быстро)
    dw = fetch_dw_articles()
    # 2) Finmarket через HTML + карточки (ограничено MAX_LINKS_FINMARKET)
    fm = fetch_finmarket_articles()
    all_items = dw + fm
    print_articles(all_items)

if __name__ == "__main__":
    main()


✅ Найдено 15 новостей (за 2 дн.)



**[1] [DW] Германия профинансирует пакет помощи Украине на 500 млн долларов**  
🗓 2025-08-14 05:19  
🔗 [https://www.dw.com/ru/германия-профинансирует-пакет-помощи-украине-на-500-млн-долларов/a-73630964?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/германия-профинансирует-пакет-помощи-украине-на-500-млн-долларов/a-73630964?maca=rus-rss-ru-all-1126-rdf)


**[2] [DW] В Берлине отреагировали на отчет Госдепа США: В Германии нет цензуры**  
🗓 2025-08-14 03:43  
🔗 [https://www.dw.com/ru/в-берлине-отреагировали-на-отчет-госдепа-сша-в-германии-нет-цензуры/a-73630463?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/в-берлине-отреагировали-на-отчет-госдепа-сша-в-германии-нет-цензуры/a-73630463?maca=rus-rss-ru-all-1126-rdf)


**[3] [DW] Трамп объявил о возможности своей встречи с Путиным и Зеленским**  
🗓 2025-08-14 00:44  
🔗 [https://www.dw.com/ru/трамп-объявил-о-возможности-своей-встречи-с-путиным-и-зеленским/a-73629967?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/трамп-объявил-о-возможности-своей-встречи-с-путиным-и-зеленским/a-73629967?maca=rus-rss-ru-all-1126-rdf)


**[4] [DW] "Маршрут Трампа" для Еревана и Баку: в Тегеране ему не очень рады?**  
🗓 2025-08-13 22:10  
🔗 [https://www.dw.com/ru/маршрут-трампа-для-еревана-и-баку-в-тегеране-ему-не-очень-рады/a-73623655?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/маршрут-трампа-для-еревана-и-баку-в-тегеране-ему-не-очень-рады/a-73623655?maca=rus-rss-ru-all-1126-rdf)


...

**[11] [DW] Зеленский прибыл в Берлин на видеоконференцию с Трампом**  
🗓 2025-08-13 17:57  
🔗 [https://www.dw.com/ru/зеленский-прибыл-в-берлин-на-видеоконференцию-с-трампом/a-73618767?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/зеленский-прибыл-в-берлин-на-видеоконференцию-с-трампом/a-73618767?maca=rus-rss-ru-all-1126-rdf)


**[12] [DW] Девочки в Германии в два раза чаще стали болеть анорексией**  
🗓 2025-08-12 19:07  
🔗 [https://www.dw.com/ru/девочки-в-германии-в-два-раза-чаще-стали-болеть-анорексией/a-73606787?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/девочки-в-германии-в-два-раза-чаще-стали-болеть-анорексией/a-73606787?maca=rus-rss-ru-all-1126-rdf)


**[13] [DW] "Голая выставка" в Германии: музей откроет двери только для обнаженных посетителей**  
🗓 2025-08-12 14:29  
🔗 [https://www.dw.com/ru/голая-выставка-в-германии-музей-откроет-двери-только-для-обнаженных-посетителей/a-73603507?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/голая-выставка-в-германии-музей-откроет-двери-только-для-обнаженных-посетителей/a-73603507?maca=rus-rss-ru-all-1126-rdf)


**[14] [DW] Вы уволены: как Трамп борется с экономической статистикой США**  
🗓 2025-08-12 12:56  
🔗 [https://www.dw.com/ru/вы-уволены-как-трамп-борется-с-экономической-статистикой-сша/a-73604373?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/вы-уволены-как-трамп-борется-с-экономической-статистикой-сша/a-73604373?maca=rus-rss-ru-all-1126-rdf)


**[15] [DW] Автопром в России заглох? Продажи рухнули, заводы в простое**  
🗓 2025-08-12 12:56  
🔗 [https://www.dw.com/ru/автопром-в-россии-заглох-продажи-рухнули-заводы-в-простое/a-73604599?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/автопром-в-россии-заглох-продажи-рухнули-заводы-в-простое/a-73604599?maca=rus-rss-ru-all-1126-rdf)


In [ ]:
import re
import html
import time
import urllib.parse
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta, timezone

import requests
from bs4 import BeautifulSoup

# ================= НАСТРОЙКИ =================
KEYWORDS = ['трамп', 'trump', 'дональд', 'сша', 'германи', 'китай', 'росси', 'эконом', 'инфляц', 'ставк']
DAYS_BACK = 2
MAX_NEWS_PER_SITE = 50
MAX_LINKS_FINMARKET = 150  # сколько карточек Finmarket открываем максимум
DEBUG = True               # включи, чтобы видеть счётчики/причины отбраковки

# DW RSS
DW_RSS_URL = "https://rss.dw.com/rdf/rss-ru-all"

# Finmarket
FINMARKET_LIST_URL = "https://www.finmarket.ru/news/"

HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/125.0 Safari/537.36"),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
}

RU_MONTHS = {
    'января': 1, 'февраля': 2, 'марта': 3, 'апреля': 4, 'мая': 5, 'июня': 6,
    'июля': 7, 'августа': 8, 'сентября': 9, 'октября': 10, 'ноября': 11, 'декабря': 12
}

ISO_REGEX = re.compile(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}(?::\d{2})?(?:Z|[+\-]\d{2}:\d{2})')
FM_LINK_RE = re.compile(
    r'href=["\'](?:https?://[^"\']+)?(/news/\d+/?)(?:\?[^"\']*)?["\']',
    re.IGNORECASE
)
RU_NUMERIC_RE = re.compile(r'(?P<d>\d{1,2})\.(?P<m>\d{1,2})\.(?P<y>\d{4})(?:\s+(?P<h>\d{1,2}):(?P<min>\d{2}))?')
RU_TEXTUAL_RE = re.compile(
    r'(?P<d>\d{1,2})\s+(?P<mon>[А-Яа-я]+)\s+(?P<y>\d{4})'
    r'(?:\s*г\.?|(?:\s*года)?)?(?:\s+(?P<h>\d{1,2}):(?P<min>\d{2}))?',
    re.IGNORECASE
)

def now_utc():
    return datetime.now(timezone.utc)

def within_days(dt: datetime, days: int) -> bool:
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return (now_utc() - dt) <= timedelta(days=days)

def title_matches(title: str) -> bool:
    tl = title.lower()
    return any(kw in tl for kw in KEYWORDS)

def parse_iso_or_rfc2822(s: str) -> datetime | None:
    if not s:
        return None
    s = s.strip()
    try:
        if s.endswith('Z'):
            dt = datetime.fromisoformat(s.replace('Z', '+00:00'))
        else:
            dt = datetime.fromisoformat(s)
        return dt.astimezone(timezone.utc)
    except Exception:
        pass
    try:
        from email.utils import parsedate_to_datetime
        dt = parsedate_to_datetime(s)
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=timezone.utc)
        return dt.astimezone(timezone.utc)
    except Exception:
        return None

def parse_russian_date(text: str) -> datetime | None:
    if not text:
        return None
    s = html.unescape(text.strip().lower())

    m = RU_NUMERIC_RE.search(s)
    if m:
        d, mth, y = int(m.group('d')), int(m.group('m')), int(m.group('y'))
        hh = int(m.group('h') or 0)
        mm = int(m.group('min') or 0)
        try:
            dt = datetime(y, mth, d, hh, mm, tzinfo=timezone(timedelta(hours=3)))
            return dt.astimezone(timezone.utc)
        except Exception:
            return None

    m = RU_TEXTUAL_RE.search(s)
    if m:
        d = int(m.group('d'))
        mon_name = m.group('mon')
        y = int(m.group('y'))
        mth = RU_MONTHS.get(mon_name, None)
        if not mth:
            return None
        hh = int(m.group('h') or 0)
        mm = int(m.group('min') or 0)
        try:
            dt = datetime(y, mth, d, hh, mm, tzinfo=timezone(timedelta(hours=3)))
            return dt.astimezone(timezone.utc)
        except Exception:
            return None
    return None

# --------- DW (RSS) ----------
def fetch_dw():
    out = []
    try:
        r = requests.get(DW_RSS_URL, headers=HEADERS, timeout=20)
        r.raise_for_status()
        root = ET.fromstring(r.content)
        items = [el for el in root.iter() if el.tag.lower().endswith('item')]
        if DEBUG: print(f"[DW] items in RSS: {len(items)}")
        for it in items:
            title = link = date_text = None
            for ch in it:
                tag = ch.tag.lower()
                if tag.endswith('title'):
                    title = (ch.text or '').strip()
                elif tag.endswith('link'):
                    link = (ch.text or '').strip()
                elif tag.endswith('date') or tag.endswith('pubdate'):
                    date_text = (ch.text or '').strip()
            if not title or not link:
                continue
            if not title_matches(title):
                continue
            dt = parse_iso_or_rfc2822(date_text)
            if not dt or not within_days(dt, DAYS_BACK):
                continue
            out.append({'source': 'DW', 'title': title, 'link': link, 'date': dt})
            if len(out) >= MAX_NEWS_PER_SITE:
                break
    except Exception as e:
        print(f"[DW][ERROR] {e}")
    return out

# --------- Finmarket ----------
def _req(url: str, timeout=20) -> requests.Response | None:
    try:
        r = requests.get(url, headers=HEADERS, timeout=timeout)
        enc = r.encoding or r.apparent_encoding or 'utf-8'
        r.encoding = enc
        if r.status_code == 200:
            return r
        if DEBUG: print(f"[Finmarket][WARN] HTTP {r.status_code} for {url}")
    except Exception as e:
        if DEBUG: print(f"[Finmarket][ERR] {e} for {url}")
    return None

def _extract_finmarket_links(list_html: str) -> list[str]:
    links = []
    seen = set()
    for m in FM_LINK_RE.finditer(list_html):
        rel = m.group(1)
        if rel and rel not in seen:
            seen.add(rel)
            links.append(urllib.parse.urljoin(FINMARKET_LIST_URL, rel))
        if len(links) >= MAX_LINKS_FINMARKET:
            break
    return links

def _iso_from_attrs(soup: BeautifulSoup) -> str | None:
    # <time datetime="2025-08-13T19:04:00+03:00"> или <meta itemprop="datePublished" content="...">
    t = soup.find('time', attrs={'datetime': True}) or soup.find('time', attrs={'content': True})
    if t:
        return t.get('datetime') or t.get('content')
    meta = soup.find('meta', attrs={'itemprop': 'datePublished'})
    if meta and meta.get('content'):
        return meta['content']
    meta2 = soup.find('meta', attrs={'property': 'article:published_time'})
    if meta2 and meta2.get('content'):
        return meta2['content']
    return None

def _extract_finmarket_date(html_text: str) -> datetime | None:
    soup = BeautifulSoup(html_text, 'html.parser')
    # 1) ISO из атрибутов
    iso = _iso_from_attrs(soup)
    if iso:
        dt = parse_iso_or_rfc2822(iso)
        if dt:
            return dt
    # 2) ISO в сыром тексте
    m = ISO_REGEX.search(html_text)
    if m:
        dt = parse_iso_or_rfc2822(m.group(0))
        if dt:
            return dt
    # 3) Русские форматы
    flat_text = ' '.join(soup.stripped_strings)
    return parse_russian_date(flat_text)

def _extract_finmarket_title(html_text: str) -> str:
    soup = BeautifulSoup(html_text, 'html.parser')
    h1 = soup.find('h1')
    if h1 and h1.get_text(strip=True):
        return h1.get_text(strip=True)
    og = soup.find('meta', attrs={'property': 'og:title'})
    if og and og.get('content'):
        return og['content'].strip()
    if soup.title and soup.title.text:
        return soup.title.text.strip()
    return ""

def fetch_finmarket():
    out = []
    list_resp = _req(FINMARKET_LIST_URL)
    if not list_resp:
        print("[Finmarket][ERROR] не удалось загрузить ленту")
        return out

    cand = _extract_finmarket_links(list_resp.text)
    if DEBUG: print(f"[Finmarket] найдено ссылок-кандидатов: {len(cand)}")

    no_title=0; no_kw=0; old_dt=0; no_dt=0
    for url in cand:
        r = _req(url, timeout=20)
        if not r:
            continue
        title = _extract_finmarket_title(r.text)
        if not title:
            no_title += 1
            continue
        if not title_matches(title):
            no_kw += 1
            continue
        dt = _extract_finmarket_date(r.text)
        if not dt:
            no_dt += 1
            continue
        if not within_days(dt, DAYS_BACK):
            old_dt += 1
            continue
        out.append({'source': 'Finmarket', 'title': title, 'link': url, 'date': dt})
        if len(out) >= MAX_NEWS_PER_SITE:
            break

    if DEBUG:
        print(f"[Finmarket] итог: {len(out)} прошло фильтр | без заголовка: {no_title}, без KEYWORDS: {no_kw}, без даты: {no_dt}, старше {DAYS_BACK} дн.: {old_dt}")
    return out

# --------- ВЫВОД ---------
def print_articles(items):
    if not items:
        print("❌ Нет подходящих новостей.")
        return
    items.sort(key=lambda x: x['date'] or datetime.min.replace(tzinfo=timezone.utc), reverse=True)
    print(f"✅ Найдено {len(items)} новостей за {DAYS_BACK} дн.\n")
    try:
        from IPython.display import display, Markdown
        for i, a in enumerate(items, 1):
            dt_local = a['date'].astimezone(timezone(timedelta(hours=6)))  # Asia/Bishkek
            display(Markdown(
                f"**[{i}] [{a['source']}] {a['title']}**  \n"
                f"🗓 {dt_local.strftime('%Y-%m-%d %H:%M')}  \n"
                f"🔗 [{a['link']}]({a['link']})\n"
            ))
    except Exception:
        for i, a in enumerate(items, 1):
            dt_local = a['date'].astimezone(timezone(timedelta(hours=6)))
            print(f"[{i}] [{a['source']}] {a['title']}\n"
                  f"🗓 {dt_local.strftime('%Y-%m-%d %H:%M')}\n"
                  f"🔗 {a['link']}\n")

def main():
    dw = fetch_dw()
    fm = fetch_finmarket()
    all_items = dw + fm
    print_articles(all_items)

if __name__ == "__main__":
    main()



[DW] items in RSS: 75
[Finmarket] найдено ссылок-кандидатов: 10
[Finmarket] итог: 0 прошло фильтр | без заголовка: 0, без KEYWORDS: 10, без даты: 0, старше 2 дн.: 0
✅ Найдено 15 новостей за 2 дн.



**[1] [DW] Германия профинансирует пакет помощи Украине на 500 млн долларов**  
🗓 2025-08-14 05:19  
🔗 [https://www.dw.com/ru/германия-профинансирует-пакет-помощи-украине-на-500-млн-долларов/a-73630964?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/германия-профинансирует-пакет-помощи-украине-на-500-млн-долларов/a-73630964?maca=rus-rss-ru-all-1126-rdf)


**[2] [DW] В Берлине отреагировали на отчет Госдепа США: В Германии нет цензуры**  
🗓 2025-08-14 03:43  
🔗 [https://www.dw.com/ru/в-берлине-отреагировали-на-отчет-госдепа-сша-в-германии-нет-цензуры/a-73630463?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/в-берлине-отреагировали-на-отчет-госдепа-сша-в-германии-нет-цензуры/a-73630463?maca=rus-rss-ru-all-1126-rdf)


**[3] [DW] Трамп объявил о возможности своей встречи с Путиным и Зеленским**  
🗓 2025-08-14 00:44  
🔗 [https://www.dw.com/ru/трамп-объявил-о-возможности-своей-встречи-с-путиным-и-зеленским/a-73629967?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/трамп-объявил-о-возможности-своей-встречи-с-путиным-и-зеленским/a-73629967?maca=rus-rss-ru-all-1126-rdf)


**[4] [DW] "Маршрут Трампа" для Еревана и Баку: в Тегеране ему не очень рады?**  
🗓 2025-08-13 22:10  
🔗 [https://www.dw.com/ru/маршрут-трампа-для-еревана-и-баку-в-тегеране-ему-не-очень-рады/a-73623655?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/маршрут-трампа-для-еревана-и-баку-в-тегеране-ему-не-очень-рады/a-73623655?maca=rus-rss-ru-all-1126-rdf)


...

**[11] [DW] Зеленский прибыл в Берлин на видеоконференцию с Трампом**  
🗓 2025-08-13 17:57  
🔗 [https://www.dw.com/ru/зеленский-прибыл-в-берлин-на-видеоконференцию-с-трампом/a-73618767?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/зеленский-прибыл-в-берлин-на-видеоконференцию-с-трампом/a-73618767?maca=rus-rss-ru-all-1126-rdf)


**[12] [DW] Девочки в Германии в два раза чаще стали болеть анорексией**  
🗓 2025-08-12 19:07  
🔗 [https://www.dw.com/ru/девочки-в-германии-в-два-раза-чаще-стали-болеть-анорексией/a-73606787?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/девочки-в-германии-в-два-раза-чаще-стали-болеть-анорексией/a-73606787?maca=rus-rss-ru-all-1126-rdf)


**[13] [DW] "Голая выставка" в Германии: музей откроет двери только для обнаженных посетителей**  
🗓 2025-08-12 14:29  
🔗 [https://www.dw.com/ru/голая-выставка-в-германии-музей-откроет-двери-только-для-обнаженных-посетителей/a-73603507?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/голая-выставка-в-германии-музей-откроет-двери-только-для-обнаженных-посетителей/a-73603507?maca=rus-rss-ru-all-1126-rdf)


**[14] [DW] Вы уволены: как Трамп борется с экономической статистикой США**  
🗓 2025-08-12 12:56  
🔗 [https://www.dw.com/ru/вы-уволены-как-трамп-борется-с-экономической-статистикой-сша/a-73604373?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/вы-уволены-как-трамп-борется-с-экономической-статистикой-сша/a-73604373?maca=rus-rss-ru-all-1126-rdf)


**[15] [DW] Автопром в России заглох? Продажи рухнули, заводы в простое**  
🗓 2025-08-12 12:56  
🔗 [https://www.dw.com/ru/автопром-в-россии-заглох-продажи-рухнули-заводы-в-простое/a-73604599?maca=rus-rss-ru-all-1126-rdf](https://www.dw.com/ru/автопром-в-россии-заглох-продажи-рухнули-заводы-в-простое/a-73604599?maca=rus-rss-ru-all-1126-rdf)


In [ ]:
import re
import html
import json
import urllib.parse
from datetime import datetime, timedelta, timezone

import requests
from bs4 import BeautifulSoup

# ================= НАСТРОЙКИ =================
KEYWORDS = [
    'трамп', 'trump', 'дональд',
    'сша', 'герман', 'китай', 'росси',
    'эконом', 'инфляц', 'ставк', 'ввп', 'moex', 'фрс', 'ecb'
]
DAYS_BACK = 2
MAX_NEWS_PER_SITE = 50           # максимум подходящих новостей на сайт (после фильтров)
MAX_LINKS_TASS = 120             # сколько карточек просматривать у TASS
MAX_LINKS_INTERFAX = 120         # сколько карточек просматривать у Interfax
DEBUG = False                    # включить диагностику отбраковки

# Источники
TASS_LIST_URL = "https://tass.ru/ekonomika"
INTERFAX_LIST_URL = "https://www.interfax.ru/business"

HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/125.0 Safari/537.36"),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
    "Referer": "https://google.com"
}

# Русские месяцы → номер
RU_MONTHS = {
    'января': 1, 'февраля': 2, 'марта': 3, 'апреля': 4, 'мая': 5, 'июня': 6,
    'июля': 7, 'августа': 8, 'сентября': 9, 'октября': 10, 'ноября': 11, 'декабря': 12
}

ISO_REGEX = re.compile(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}(?::\d{2})?(?:Z|[+\-]\d{2}:\d{2})', re.IGNORECASE)

# ================= ВСПОМОГАТЕЛЬНОЕ =================
def now_utc():
    return datetime.now(timezone.utc)

def within_days(dt: datetime, days: int) -> bool:
    """dt должен быть tz-aware; сравнение с текущим UTC."""
    if dt.tzinfo is None:
        dt = dt.replace(tzinfo=timezone.utc)
    return (now_utc() - dt) <= timedelta(days=days)

def title_matches(title: str) -> bool:
    tl = (title or "").lower()
    return any(kw in tl for kw in KEYWORDS)

def _req(url: str, timeout=20) -> requests.Response | None:
    try:
        r = requests.get(url, headers=HEADERS, timeout=timeout)
        enc = r.encoding or r.apparent_encoding or 'utf-8'
        r.encoding = enc
        if r.status_code == 200:
            return r
        if DEBUG: print(f"[WARN] HTTP {r.status_code} for {url}")
    except Exception as e:
        if DEBUG: print(f"[ERR] {e} for {url}")
    return None

# -------- парсинг дат --------
def parse_iso_or_rfc2822(s: str) -> datetime | None:
    if not s:
        return None
    s = s.strip()
    # ISO 8601
    try:
        if s.endswith('Z'):
            dt = datetime.fromisoformat(s.replace('Z', '+00:00'))
        else:
            dt = datetime.fromisoformat(s)
        return dt.astimezone(timezone.utc)
    except Exception:
        pass
    # RFC2822
    try:
        from email.utils import parsedate_to_datetime
        dt = parsedate_to_datetime(s)
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=timezone.utc)
        return dt.astimezone(timezone.utc)
    except Exception:
        return None

RU_NUMERIC_RE = re.compile(
    r'(?P<d>\d{1,2})\.(?P<m>\d{1,2})\.(?P<y>\d{4})(?:\s+(?P<h>\d{1,2}):(?P<min>\d{2}))?'
)
RU_TEXTUAL_RE = re.compile(
    r'(?P<d>\d{1,2})\s+(?P<mon>[А-Яа-я]+)\s+(?P<y>\d{4})'
    r'(?:\s*г\.?|(?:\s*года)?)?(?:,\s*|\s+)?(?:(?P<h>\d{1,2}):(?P<min>\d{2}))?',
    re.IGNORECASE
)

def parse_russian_date(text: str, default_tz=timezone(timedelta(hours=3))) -> datetime | None:
    """Парсим '13 августа 2025, 16:49' или '13.08.2025 16:49'. Возвращаем UTC."""
    if not text:
        return None
    s = html.unescape(text.strip().lower())

    m = RU_NUMERIC_RE.search(s)
    if m:
        d, mth, y = int(m.group('d')), int(m.group('m')), int(m.group('y'))
        hh = int(m.group('h') or 0)
        mm = int(m.group('min') or 0)
        try:
            dt = datetime(y, mth, d, hh, mm, tzinfo=default_tz)
            return dt.astimezone(timezone.utc)
        except Exception:
            return None

    m = RU_TEXTUAL_RE.search(s)
    if m:
        d = int(m.group('d'))
        mon_name = m.group('mon')
        y = int(m.group('y'))
        mth = RU_MONTHS.get(mon_name, None)
        if not mth:
            return None
        hh = int(m.group('h') or 0)
        mm = int(m.group('min') or 0)
        try:
            dt = datetime(y, mth, d, hh, mm, tzinfo=default_tz)
            return dt.astimezone(timezone.utc)
        except Exception:
            return None
    return None

def extract_jsonld_datetime(soup: BeautifulSoup) -> datetime | None:
    """Читаем datePublished из JSON-LD (NewsArticle)."""
    for tag in soup.find_all("script", attrs={"type": "application/ld+json"}):
        try:
            data = json.loads(tag.string or tag.text or "")
        except Exception:
            continue
        objs = data if isinstance(data, list) else [data]
        for obj in objs:
            if not isinstance(obj, dict):
                continue
            dp = obj.get("datePublished") or obj.get("dateCreated")
            if dp:
                dt = parse_iso_or_rfc2822(dp)
                if dt:
                    return dt
    return None

def extract_datetime_generic(html_text: str, soup: BeautifulSoup) -> datetime | None:
    # 1) JSON-LD
    dt = extract_jsonld_datetime(soup)
    if dt:
        return dt
    # 2) <time datetime="..."> или <meta itemprop="datePublished" content="..."> или og:published_time
    t = soup.find('time', attrs={'datetime': True}) or soup.find('time', attrs={'content': True})
    if t:
        dt = parse_iso_or_rfc2822(t.get('datetime') or t.get('content'))
        if dt:
            return dt
    meta = soup.find('meta', attrs={'itemprop': 'datePublished'}) or \
           soup.find('meta', attrs={'property': 'article:published_time'})
    if meta and meta.get('content'):
        dt = parse_iso_or_rfc2822(meta['content'])
        if dt:
            return dt
    # 3) ISO в сыром HTML
    m = ISO_REGEX.search(html_text)
    if m:
        dt = parse_iso_or_rfc2822(m.group(0))
        if dt:
            return dt
    # 4) Русский текст (берём весь текст страницы)
    flat = ' '.join(soup.stripped_strings)
    return parse_russian_date(flat)

def extract_title_generic(soup: BeautifulSoup) -> str:
    h1 = soup.find('h1')
    if h1 and h1.get_text(strip=True):
        return h1.get_text(strip=True)
    og = soup.find('meta', attrs={'property': 'og:title'})
    if og and og.get('content'):
        return og['content'].strip()
    if soup.title and soup.title.text:
        return soup.title.text.strip()
    return ""

# ================= TASS =================
# Ловим ссылки статей раздела /ekonomika/<id>...
TASS_LINK_RE = re.compile(
    r'href=["\'](?:https?://(?:www\.)?tass\.ru)?(/ekonomika/\d+[^\s"\']*)["\']',
    re.IGNORECASE
)

def fetch_tass():
    out = []
    list_resp = _req(TASS_LIST_URL)
    if not list_resp:
        print("[TASS][ERROR] не удалось загрузить ленту")
        return out

    html_list = list_resp.text
    links = []
    seen = set()
    for m in TASS_LINK_RE.finditer(html_list):
        rel = m.group(1)
        if not rel:
            continue
        url = urllib.parse.urljoin(TASS_LIST_URL, rel)
        if url not in seen:
            seen.add(url)
            links.append(url)
        if len(links) >= MAX_LINKS_TASS:
            break

    if DEBUG:
        print(f"[TASS] кандидатов: {len(links)}")

    no_title = no_kw = no_dt = old_dt = 0
    for url in links:
        r = _req(url, timeout=20)
        if not r:
            continue
        soup = BeautifulSoup(r.text, "html.parser")
        title = extract_title_generic(soup)
        if not title:
            no_title += 1
            continue
        if not title_matches(title):
            no_kw += 1
            continue
        dt = extract_datetime_generic(r.text, soup)
        if not dt:
            no_dt += 1
            continue
        if not within_days(dt, DAYS_BACK):
            old_dt += 1
            continue
        out.append({'source': 'TASS', 'title': title, 'link': url, 'date': dt})
        if len(out) >= MAX_NEWS_PER_SITE:
            break

    if DEBUG:
        print(f"[TASS] прошло фильтр: {len(out)} | без заголовка: {no_title}, без KEYWORDS: {no_kw}, без даты: {no_dt}, старше {DAYS_BACK} дн.: {old_dt}")
    return out

# ================= Interfax =================
# Ссылки вида /business/123456
INTERFAX_LINK_RE = re.compile(
    r'href=["\'](?:https?://(?:www\.)?interfax\.ru)?(/business/\d+[^\s"\']*)["\']',
    re.IGNORECASE
)

def fetch_interfax():
    out = []
    list_resp = _req(INTERFAX_LIST_URL)
    if not list_resp:
        print("[Interfax][ERROR] не удалось загрузить ленту")
        return out

    html_list = list_resp.text
    links = []
    seen = set()
    for m in INTERFAX_LINK_RE.finditer(html_list):
        rel = m.group(1)
        if not rel:
            continue
        url = urllib.parse.urljoin(INTERFAX_LIST_URL, rel)
        if url not in seen:
            seen.add(url)
            links.append(url)
        if len(links) >= MAX_LINKS_INTERFAX:
            break

    if DEBUG:
        print(f"[Interfax] кандидатов: {len(links)}")

    no_title = no_kw = no_dt = old_dt = 0
    for url in links:
        r = _req(url, timeout=20)
        if not r:
            continue
        soup = BeautifulSoup(r.text, "html.parser")
        title = extract_title_generic(soup)
        if not title:
            no_title += 1
            continue
        if not title_matches(title):
            no_kw += 1
            continue
        dt = extract_datetime_generic(r.text, soup)
        if not dt:
            no_dt += 1
            continue
        if not within_days(dt, DAYS_BACK):
            old_dt += 1
            continue
        out.append({'source': 'Interfax', 'title': title, 'link': url, 'date': dt})
        if len(out) >= MAX_NEWS_PER_SITE:
            break

    if DEBUG:
        print(f"[Interfax] прошло фильтр: {len(out)} | без заголовка: {no_title}, без KEYWORDS: {no_kw}, без даты: {no_dt}, старше {DAYS_BACK} дн.: {old_dt}")
    return out

# ================= ВЫВОД =================
def print_articles(items):
    if not items:
        print("❌ Нет подходящих новостей.")
        return
    items.sort(key=lambda x: x['date'] or datetime.min.replace(tzinfo=timezone.utc), reverse=True)
    print(f"✅ Найдено {len(items)} новостей за {DAYS_BACK} дн.\n")

    # Jupyter: кликабельные ссылки
    try:
        from IPython.display import display, Markdown
        for i, a in enumerate(items, 1):
            # выводим по Asia/Bishkek (UTC+6), чтобы было в твоей TZ
            dt_local = a['date'].astimezone(timezone(timedelta(hours=6)))
            display(Markdown(
                f"**[{i}] [{a['source']}] {a['title']}**  \n"
                f"🗓 {dt_local.strftime('%Y-%m-%d %H:%M')}  \n"
                f"🔗 [{a['link']}]({a['link']})\n"
            ))
    except Exception:
        for i, a in enumerate(items, 1):
            dt_local = a['date'].astimezone(timezone(timedelta(hours=6)))
            print(f"[{i}] [{a['source']}] {a['title']}\n"
                  f"🗓 {dt_local.strftime('%Y-%m-%d %H:%M')}\n"
                  f"🔗 {a['link']}\n")

def main():
    tass_items = fetch_tass()
    interfax_items = fetch_interfax()
    all_items = tass_items + interfax_items
    print_articles(all_items)

if __name__ == "__main__":
    main()


✅ Найдено 4 новостей за 2 дн.



**[1] [TASS] Аксаков: с 2026 года россиянам запретят брать более одного займа в МФО**  
🗓 2025-08-14 07:34  
🔗 [https://tass.ru/ekonomika/24776439](https://tass.ru/ekonomika/24776439)


**[2] [TASS] В России отменили плановые проверки аудиторских организаций среднего риска**  
🗓 2025-08-14 01:18  
🔗 [https://tass.ru/ekonomika/24775349](https://tass.ru/ekonomika/24775349)


**[3] [TASS] Бразилия для ответа на тарифы США организует переговоры членов БРИКС**  
🗓 2025-08-14 01:03  
🔗 [https://tass.ru/ekonomika/24775379](https://tass.ru/ekonomika/24775379)


**[4] [TASS] Минфин США до 20 августа разрешил транзакции с РФ, необходимые для саммита**  
🗓 2025-08-14 00:25  
🔗 [https://tass.ru/ekonomika/24775275](https://tass.ru/ekonomika/24775275)


In [ ]:
Готово. Я собрал для тебя **полноценный Jupyter Notebook** + **эквивалентный .py-скрипт** со всем пайплайном (загрузка, выравнивание, корреляции в уровнях и разностях, rolling/cross-corr, детекция шоков, наглядные графики с шейдингом, расчёт приближённых bond returns и корреляции в shock-периодах).

### Файлы

* **Ноутбук:** [ust\_infl\_yield\_fed\_correlation\_analysis.ipynb](sandbox:/mnt/data/ust_infl_yield_fed_correlation_analysis.ipynb)
* **Скрипт:** [analysis.py](sandbox:/mnt/data/analysis.py)
* **Датасет (синтезированный, пока не подложишь свои CSV):** [ust\_infl\_ffr\_dataset.csv](sandbox:/mnt/data/ust_infl_ffr_dataset.csv)
* **Возвраты облигаций (duration × Δy):** [approx\_returns.csv](sandbox:/mnt/data/approx_returns.csv)
* **Корреляции по shock-окнам:** [correlations\_shock\_windows.csv](sandbox:/mnt/data/correlations_shock_windows.csv)
* Плюс все промежуточные таблицы (corr\_infl\_vs\_yields\_*.csv, rolling\_corr\_*.csv, crosscorr\_\*.csv) — уже сохраняются в `/mnt/data/`.

### Что внутри ноутбука

1. **Загрузка данных** из `/mnt/data/*.csv` (auto-detect колонок). Если файлов нет — генерируется синтетика, чтобы код работал end-to-end.
2. **Выравнивание к month-end**, расчёт **Δ** и наклонов кривой (slope\_30\_5, slope\_10\_5).
3. **Корреляции**: уровни и Δ между `infl_exp`, `fed_funds` и доходностями `y5/y10/y20/y30`.
4. **Rolling corr (36m)** для infl ↔ каждой дюрации.
5. **Cross-correlation** (lead/lag −12…+12) для infl↔yields и ΔFed↔Δyields.
6. **Shock detection**: окна шоков по |Δ|≥95-й перцентиль (расширены на ±1 месяц), **шейдинг** на графиках:

   * *Inflation expectations + UST yields*
   * *Fed Funds + UST yields*
7. **Scatter-диаграммы**: yields vs infl / Δy vs Δinfl / yields vs Fed Funds (с corr в заголовках).
8. **Bond returns (approx)** через `return ≈ −Duration × Δyield` и **корреляции в shock-окнах**:

   * (Δinfl\_exp, returns\_{5,10,20,30}) в инфляционных шоках;
   * (Δfed\_funds, returns\_{5,10,20,30}) в шоках ставки.
9. Блок «**Optional: OLS (Newey–West)**» — закомментирован, при наличии `statsmodels` можно включить и получить формальные β на разных дюрациях и Wald-тест равенства β(30Y) и β(5Y).

### Как запустить на **твоих** данных

Положи файлы в `/mnt/data/` и просто открой/запусти ноутбук:

* `inflation_expectations.csv` — колонки `date, infl_exp` (или `value/breakeven/expectations`)
* `treasury_yields.csv` — `date, y5, y10, y20, y30` (или `DGS5,DGS10,DGS20,DGS30`)
* `fed_funds.csv` — `date, fed_funds` (или `DFF/EFFR/fedrate/ffr`)

Код сам выровняет частоту, пересчитает метрики и перерисует все графики с выделением шоковых таймфреймов.

Если хочешь — добавлю в ноутбук:

* точные **total returns** с купоном и convexity (вместо приближения duration×Δy),
* регрессии с **Newey–West** и сравнение чувствительности правого vs левого края (Wald-тест),
* **Local Projections** (Jordà) для импульс-реакций на горизонтах 1–12 мес.


Готово. Я собрал для тебя **полноценный Jupyter Notebook** + **эквивалентный .py-скрипт** со всем пайплайном (загрузка, выравнивание, корреляции в уровнях и разностях, rolling/cross-corr, детекция шоков, наглядные графики с шейдингом, расчёт приближённых bond returns и корреляции в shock-периодах).

### Файлы

* **Ноутбук:** [ust\_infl\_yield\_fed\_correlation\_analysis.ipynb](sandbox:/mnt/data/ust_infl_yield_fed_correlation_analysis.ipynb)
* **Скрипт:** [analysis.py](sandbox:/mnt/data/analysis.py)
* **Датасет (синтезированный, пока не подложишь свои CSV):** [ust\_infl\_ffr\_dataset.csv](sandbox:/mnt/data/ust_infl_ffr_dataset.csv)
* **Возвраты облигаций (duration × Δy):** [approx\_returns.csv](sandbox:/mnt/data/approx_returns.csv)
* **Корреляции по shock-окнам:** [correlations\_shock\_windows.csv](sandbox:/mnt/data/correlations_shock_windows.csv)
* Плюс все промежуточные таблицы (corr\_infl\_vs\_yields\_*.csv, rolling\_corr\_*.csv, crosscorr\_\*.csv) — уже сохраняются в `/mnt/data/`.

### Что внутри ноутбука

1. **Загрузка данных** из `/mnt/data/*.csv` (auto-detect колонок). Если файлов нет — генерируется синтетика, чтобы код работал end-to-end.
2. **Выравнивание к month-end**, расчёт **Δ** и наклонов кривой (slope\_30\_5, slope\_10\_5).
3. **Корреляции**: уровни и Δ между `infl_exp`, `fed_funds` и доходностями `y5/y10/y20/y30`.
4. **Rolling corr (36m)** для infl ↔ каждой дюрации.
5. **Cross-correlation** (lead/lag −12…+12) для infl↔yields и ΔFed↔Δyields.
6. **Shock detection**: окна шоков по |Δ|≥95-й перцентиль (расширены на ±1 месяц), **шейдинг** на графиках:

   * *Inflation expectations + UST yields*
   * *Fed Funds + UST yields*
7. **Scatter-диаграммы**: yields vs infl / Δy vs Δinfl / yields vs Fed Funds (с corr в заголовках).
8. **Bond returns (approx)** через `return ≈ −Duration × Δyield` и **корреляции в shock-окнах**:

   * (Δinfl\_exp, returns\_{5,10,20,30}) в инфляционных шоках;
   * (Δfed\_funds, returns\_{5,10,20,30}) в шоках ставки.
9. Блок «**Optional: OLS (Newey–West)**» — закомментирован, при наличии `statsmodels` можно включить и получить формальные β на разных дюрациях и Wald-тест равенства β(30Y) и β(5Y).

### Как запустить на **твоих** данных

Положи файлы в `/mnt/data/` и просто открой/запусти ноутбук:

* `inflation_expectations.csv` — колонки `date, infl_exp` (или `value/breakeven/expectations`)
* `treasury_yields.csv` — `date, y5, y10, y20, y30` (или `DGS5,DGS10,DGS20,DGS30`)
* `fed_funds.csv` — `date, fed_funds` (или `DFF/EFFR/fedrate/ffr`)

Код сам выровняет частоту, пересчитает метрики и перерисует все графики с выделением шоковых таймфреймов.

Если хочешь — добавлю в ноутбук:

* точные **total returns** с купоном и convexity (вместо приближения duration×Δy),
* регрессии с **Newey–West** и сравнение чувствительности правого vs левого края (Wald-тест),
* **Local Projections** (Jordà) для импульс-реакций на горизонтах 1–12 мес.


Отлично, вот полный, самодостаточный Python-скрипт (один файл), который повторяет весь пайплайн: загрузка данных (или синтетика, если файлов нет), выравнивание к month-end, корреляции (levels/Δ), rolling & cross-corr, выделение shock-окон, графики с шейдингом, приближённые bond returns через duration×Δy и корреляции (Δinfl_exp, returns) и (Δfed_funds, returns) в соответствующих окнах.
Сохраняет все таблицы и графики в папку ./outputs.

Скопируй это в файл analysis.py и запусти python analysis.py.
Если положишь свои CSV в корень проекта — код их подхватит автоматически.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Inflation expectations, Treasury yields (5Y/10Y/20Y/30Y), and Fed Funds — correlation & shock analysis.
Author: ChatGPT (for Daytook)

Запуск:
    python analysis.py

Опционально: положи реальные CSV рядом со скриптом:
    - inflation_expectations.csv  (колонки: date, infl_exp  | или value/breakeven/expectations)
    - treasury_yields.csv         (колонки: date, y5, y10, y20, y30 | или DGS5,DGS10,DGS20,DGS30)
    - fed_funds.csv               (колонки: date, fed_funds | или DFF,EFFR,fedrate,ffr)

Если файлов нет — сгенерируется реалистичная синтетика (для корректной работы пайплайна).
"""

import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- Настройки вывода ----
OUT_DIR = Path("./outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

INFL_CSV = Path("./inflation_expectations.csv")
UST_CSV  = Path("./treasury_yields.csv")
FFR_CSV  = Path("./fed_funds.csv")

plt.rcParams["figure.figsize"] = (10, 5)  # без seaborn, без кастомных цветов


# =========================
# Helpers
# =========================
def parse_date_col(df: pd.DataFrame) -> pd.DatetimeIndex:
    for cand in ["date", "Date", "DATE", "observation_date", "period", "time"]:
        if cand in df.columns:
            return pd.to_datetime(df[cand])
    if isinstance(df.index, pd.DatetimeIndex):
        return df.index
    raise ValueError("Не найден столбец даты. Добавь 'date' или поставь DatetimeIndex.")

def pick_first_present(df: pd.DataFrame, names: list[str]) -> pd.Series:
    for n in names:
        if n in df.columns:
            return df[n].astype(float)
    raise KeyError(f"Нет ни одной из колонок {names}. Найдено: {list(df.columns)}")

def to_month_end(s: pd.Series) -> pd.Series:
    s = s.sort_index()
    sm = s.resample("M").last()
    return sm.ffill(limit=2)

def load_or_fake_data(infl_csv: Path, ust_csv: Path, ffr_csv: Path) -> pd.DataFrame:
    """Грузим реальные CSV, иначе синтетика с 2005-01 по текущий месяц."""
    start = "2005-01-01"
    idxM = pd.date_range(start, pd.Timestamp.today().strftime("%Y-%m-01"), freq="M")
    rng = np.random.default_rng(42)

    # --- синтетика (fallback) ---
    infl_syn = pd.Series(
        2.5 + 0.3*np.sin(np.linspace(0, 25, len(idxM))) + 0.5*rng.normal(size=len(idxM)),
        index=idxM, name="infl_exp"
    ).clip(0, None)

    y5_syn  = pd.Series(2.2 + 0.5*infl_syn/2.5 + 0.3*rng.normal(size=len(idxM)), index=idxM, name="y5").clip(0, None)
    y10_syn = pd.Series(y5_syn + 0.2 + 0.1*np.sin(np.linspace(0, 10, len(idxM))) + 0.2*rng.normal(size=len(idxM)),
                        index=idxM, name="y10").clip(0, None)
    y20_syn = pd.Series(y10_syn + 0.15 + 0.15*rng.normal(size=len(idxM)), index=idxM, name="y20").clip(0, None)
    y30_syn = pd.Series(y10_syn + 0.25 + 0.2*rng.normal(size=len(idxM)), index=idxM, name="y30").clip(0, None)
    ffr_syn = pd.Series((1.0 + 0.6*infl_syn + 0.3*rng.normal(size=len(idxM))).clip(0, None),
                        index=idxM, name="fed_funds")

    have_files = INFL_CSV.exists() and UST_CSV.exists() and FFR_CSV.exists()
    if have_files:
        print("[INFO] Загружаю пользовательские CSV...")
        infl_df = pd.read_csv(infl_csv)
        ust_df  = pd.read_csv(ust_csv)
        ffr_df  = pd.read_csv(ffr_csv)

        infl = pd.Series(
            pick_first_present(infl_df, ["infl_exp", "inflation", "value", "breakeven", "expectations"]).values,
            index=parse_date_col(infl_df), name="infl_exp"
        )
        infl = to_month_end(infl)

        ust_df.index = parse_date_col(ust_df)
        y5  = to_month_end(pick_first_present(ust_df, ["y5", "DGS5", "5y", "UST5Y"]))
        y10 = to_month_end(pick_first_present(ust_df, ["y10", "DGS10", "10y", "UST10Y"]))
        y20 = to_month_end(pick_first_present(ust_df, ["y20", "DGS20", "20y", "UST20Y"]))
        y30 = to_month_end(pick_first_present(ust_df, ["y30", "DGS30", "30y", "UST30Y"]))

        ffr = pd.Series(
            pick_first_present(ffr_df, ["fed_funds", "DFF", "EFFR", "fedrate", "ffr"]).values,
            index=parse_date_col(ffr_df), name="fed_funds"
        )
        ffr = to_month_end(ffr)
    else:
        print("[WARN] CSV не найдены → использую синтетические ряды для демонстрации.")
        infl, y5, y10, y20, y30, ffr = infl_syn, y5_syn, y10_syn, y20_syn, y30_syn, ffr_syn

    df = pd.concat([infl, y5, y10, y20, y30, ffr], axis=1).dropna()
    df = df[df.index >= "2005-01-31"]
    return df

def cross_corr(x: pd.Series, y: pd.Series, max_lag: int = 12) -> pd.Series:
    lags = list(range(-max_lag, max_lag + 1))
    x, y = x - x.mean(), y - y.mean()
    out = []
    for k in lags:
        if k > 0:
            v = x.shift(k).corr(y)
        elif k < 0:
            v = x.corr(y.shift(-k))
        else:
            v = x.corr(y)
        out.append(v)
    return pd.Series(out, index=lags)

def expand_windows(shock_months: pd.DatetimeIndex) -> list[tuple[pd.Timestamp, pd.Timestamp]]:
    """Окно = [предыдущий месяц .. следующий месяц] вокруг шока + мердж пересечений."""
    win = []
    for dt in shock_months:
        start = (dt - pd.offsets.MonthBegin(1)).normalize()
        end   = (dt + pd.offsets.MonthEnd(1)).normalize()
        win.append([start, end])
    win.sort(key=lambda x: x[0])
    merged = []
    for s, e in win:
        if not merged or s > merged[-1][1]:
            merged.append([s, e])
        else:
            merged[-1][1] = max(merged[-1][1], e)
    return [(s, e) for s, e in merged]


# =========================
# Основной пайплайн
# =========================
def main():
    df = load_or_fake_data(INFL_CSV, UST_CSV, FFR_CSV)
    df.to_csv(OUT_DIR / "ust_infl_ffr_dataset.csv", index_label="date")
    print(f"[OK] Dataset saved → {OUT_DIR/'ust_infl_ffr_dataset.csv'}")

    mats = ["y5", "y10", "y20", "y30"]

    # Δ и наклоны
    d = df.diff().dropna().rename(columns=lambda c: c + "_diff")
    dat = pd.concat([df, d], axis=1).dropna()
    dat["slope_30_5"] = dat["y30"] - dat["y5"]
    dat["slope_10_5"] = dat["y10"] - dat["y5"]

    # ---------- Корреляции (levels / diffs) ----------
    corr_levels = dat[["infl_exp"] + mats].corr().loc[["infl_exp"], mats]
    corr_diffs  = dat[["infl_exp_diff"] + [m+"_diff" for m in mats]].corr().loc[["infl_exp_diff"], [m+"_diff" for m in mats]]
    corr_ff_levels = dat[["fed_funds"] + mats].corr().loc[["fed_funds"], mats]
    corr_ff_diffs  = dat[["fed_funds_diff"] + [m+"_diff" for m in mats]].corr().loc[["fed_funds_diff"], [m+"_diff" for m in mats]]

    corr_levels.to_csv(OUT_DIR / "corr_infl_vs_yields_levels.csv")
    corr_diffs.to_csv(OUT_DIR / "corr_infl_vs_yields_diffs.csv")
    corr_ff_levels.to_csv(OUT_DIR / "corr_fedfunds_vs_yields_levels.csv")
    corr_ff_diffs.to_csv(OUT_DIR / "corr_fedfunds_vs_yields_diffs.csv")

    print("[OK] Static correlations saved.")

    # ---------- Rolling 36m corr: infl vs yields ----------
    window = 36
    roll = pd.DataFrame({m: dat["infl_exp"].rolling(window).corr(dat[m]) for m in mats})
    roll.to_csv(OUT_DIR / "rolling_corr_infl_vs_yields_levels_36m.csv")

    for m in mats:
        plt.figure()
        plt.plot(roll.index, roll[m].values)
        plt.title(f"Rolling {window}m corr: Inflation vs {m.upper()} (levels)")
        plt.xlabel("Date"); plt.ylabel("Correlation"); plt.grid(True, linestyle="--", alpha=0.6)
        plt.tight_layout(); plt.savefig(OUT_DIR / f"rolling_corr_infl_{m}.png", dpi=150)
        plt.close()

    # ---------- Cross-correlation (lead/lag) ----------
    cc_infl = pd.DataFrame({m: cross_corr(dat["infl_exp"], dat[m], 12) for m in mats})
    cc_infl.to_csv(OUT_DIR / "crosscorr_infl_vs_yields_levels_lag12.csv")
    for m in mats:
        cc = cc_infl[m]
        plt.figure()
        plt.plot(cc.index, cc.values, marker="o")
        plt.axhline(0, linestyle="--")
        plt.title(f"Cross-corr: Inflation vs {m.upper()} (levels)")
        plt.xlabel("Lag (months) — positive: inflation leads"); plt.ylabel("Correlation")
        plt.grid(True, linestyle="--", alpha=0.6); plt.tight_layout()
        plt.savefig(OUT_DIR / f"crosscorr_infl_{m}.png", dpi=150)
        plt.close()

    cc_ff = pd.DataFrame({m+"_diff": cross_corr(dat["fed_funds_diff"], dat[m+"_diff"], 12) for m in mats})
    cc_ff.to_csv(OUT_DIR / "crosscorr_fedfundsdiff_vs_yieldsdiff_lag12.csv")
    for m in mats:
        cc = cc_ff[m+"_diff"]
        plt.figure()
        plt.plot(cc.index, cc.values, marker="o")
        plt.axhline(0, linestyle="--")
        plt.title(f"Cross-corr: ΔFed Funds vs Δ{m.upper()}")
        plt.xlabel("Lag (months) — positive: ΔFed leads"); plt.ylabel("Correlation")
        plt.grid(True, linestyle="--", alpha=0.6); plt.tight_layout()
        plt.savefig(OUT_DIR / f"crosscorr_fed_{m}.png", dpi=150)
        plt.close()

    # ---------- Shock detection ----------
    infl_thr = dat["infl_exp_diff"].abs().quantile(0.95)
    rate_thr = dat["fed_funds_diff"].abs().quantile(0.95)
    infl_shock_months = dat.index[dat["infl_exp_diff"].abs() >= infl_thr]
    rate_shock_months = dat.index[dat["fed_funds_diff"].abs() >= rate_thr]
    infl_windows = expand_windows(infl_shock_months)
    rate_windows = expand_windows(rate_shock_months)

    # сохранить окна в CSV
    pd.DataFrame(infl_windows, columns=["start", "end"]).to_csv(OUT_DIR / "inflation_shock_windows.csv", index=False)
    pd.DataFrame(rate_windows, columns=["start", "end"]).to_csv(OUT_DIR / "rate_shock_windows.csv", index=False)

    # ---------- Линейные графики с шейдингом ----------
    # Infl + yields
    plt.figure(figsize=(12, 5))
    plt.plot(df.index, df["infl_exp"], label="Inflation exp.")
    for m in mats:
        plt.plot(df.index, df[m], label=m.upper())
    for s, e in infl_windows:
        plt.axvspan(s, e, alpha=0.15)
    plt.title("Inflation expectations & UST yields — shaded: inflation-shock windows")
    plt.xlabel("Date"); plt.ylabel("Level (%)"); plt.grid(True, linestyle="--", alpha=0.6); plt.legend()
    plt.tight_layout(); plt.savefig(OUT_DIR / "inflation_vs_yields_shaded.png", dpi=170)
    plt.close()

    # Fed Funds + yields
    plt.figure(figsize=(12, 5))
    plt.plot(df.index, df["fed_funds"], label="Fed Funds")
    for m in mats:
        plt.plot(df.index, df[m], label=m.upper())
    for s, e in rate_windows:
        plt.axvspan(s, e, alpha=0.15)
    plt.title("Fed Funds & UST yields — shaded: rate-shock windows")
    plt.xlabel("Date"); plt.ylabel("Level (%)"); plt.grid(True, linestyle="--", alpha=0.6); plt.legend()
    plt.tight_layout(); plt.savefig(OUT_DIR / "fedfunds_vs_yields_shaded.png", dpi=170)
    plt.close()

    # ---------- Scatter: yields vs inflation (levels & diffs) ----------
    for m in mats:
        plt.figure(); plt.scatter(df["infl_exp"], df[m], s=10)
        corr = df[m].corr(df["infl_exp"])
        plt.title(f"{m.upper()} vs Inflation (levels) — corr={corr:.2f}")
        plt.xlabel("Inflation expectations (%)"); plt.ylabel(f"{m.upper()} yield (%)")
        plt.grid(True, linestyle="--", alpha=0.6); plt.tight_layout()
        plt.savefig(OUT_DIR / f"scatter_levels_{m}_infl.png", dpi=150); plt.close()

    for m in mats:
        plt.figure(); plt.scatter(dat["infl_exp_diff"], dat[m+"_diff"], s=10)
        corr = dat[m+"_diff"].corr(dat["infl_exp_diff"])
        plt.title(f"Δ{m.upper()} vs ΔInflation — corr={corr:.2f}")
        plt.xlabel("ΔInflation (%)"); plt.ylabel(f"Δ{m.upper()} (%)")
        plt.grid(True, linestyle="--", alpha=0.6); plt.tight_layout()
        plt.savefig(OUT_DIR / f"scatter_diffs_{m}_infl.png", dpi=150); plt.close()

    # ---------- Scatter: yields vs Fed Funds (levels) ----------
    for m in mats:
        plt.figure(); plt.scatter(df["fed_funds"], df[m], s=10)
        corr = df[m].corr(df["fed_funds"])
        plt.title(f"{m.upper()} vs Fed Funds (levels) — corr={corr:.2f}")
        plt.xlabel("Fed Funds (%)"); plt.ylabel(f"{m.upper()} yield (%)")
        plt.grid(True, linestyle="--", alpha=0.6); plt.tight_layout()
        plt.savefig(OUT_DIR / f"scatter_levels_{m}_fed.png", dpi=150); plt.close()

    # ---------- Approx bond returns (duration×Δy) и корреляции в shock-окнах ----------
    dur = {"y5": 4.7, "y10": 8.7, "y20": 13.5, "y30": 18.0}
    rets = pd.DataFrame(index=dat.index)
    for m in mats:
        dy_dec = dat[m + "_diff"] / 100.0
        rets[m.replace("y", "ret_")] = -dur[m] * dy_dec
    rets.to_csv(OUT_DIR / "approx_returns.csv", index_label="date")

    def corr_in_windows(x: pd.Series, y: pd.Series, windows: list[tuple[pd.Timestamp, pd.Timestamp]]):
        mask = pd.Series(False, index=x.index)
        for s, e in windows:
            mask |= ((x.index >= s) & (x.index <= e))
        xw, yw = x[mask], y[mask]
        return float("nan") if len(xw) < 3 else xw.corr(yw)

    rows = []
    # (Δinfl_exp, returns_m) в окнах инфляционных шоков
    for m in mats:
        ret_col = m.replace("y", "ret_")
        r = corr_in_windows(dat["infl_exp_diff"], rets[ret_col], infl_windows)
        rows.append({"window": "inflation_shocks", "pair": f"Δinfl_exp vs {ret_col}", "corr": r})
    # (Δfed_funds, returns_m) в окнах шоков ставки
    for m in mats:
        ret_col = m.replace("y", "ret_")
        r = corr_in_windows(dat["fed_funds_diff"], rets[ret_col], rate_windows)
        rows.append({"window": "rate_shocks", "pair": f"Δfed_funds vs {ret_col}", "corr": r})

    corr_shocks = pd.DataFrame(rows)
    corr_shocks.to_csv(OUT_DIR / "correlations_shock_windows.csv", index=False)

    # ---------- Итоговая сводка правый vs левый край ----------
    summary = pd.DataFrame({
        "infl_vs_y5_level":   [corr_levels.loc["infl_exp", "y5"]],
        "infl_vs_y30_level":  [corr_levels.loc["infl_exp", "y30"]],
        "infl_diff_vs_y5":    [corr_diffs.loc["infl_exp_diff", "y5_diff"]],
        "infl_diff_vs_y30":   [corr_diffs.loc["infl_exp_diff", "y30_diff"]],
        "fed_diff_vs_y5":     [corr_ff_diffs.loc["fed_funds_diff", "y5_diff"]],
        "fed_diff_vs_y30":    [corr_ff_diffs.loc["fed_funds_diff", "y30_diff"]],
        "slope_30_5_vs_infl": [dat[["slope_30_5","infl_exp"]].corr().iloc[0,1]],
    })
    summary["right_minus_left_infl_level"] = summary["infl_vs_y30_level"] - summary["infl_vs_y5_level"]
    summary["right_minus_left_infl_diff"]  = summary["infl_diff_vs_y30"]   - summary["infl_diff_vs_y5"]
    summary["right_minus_left_fed_diff"]   = summary["fed_diff_vs_y30"]    - summary["fed_diff_vs_y5"]
    summary.to_csv(OUT_DIR / "summary_right_vs_left.csv", index=False)

    print("[DONE] Все таблицы и графики сохранены в ./outputs")
    print(summary.round(3))

if __name__ == "__main__":
    main()


/tmp/ipython-input-1638783889.py:60: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  idxM = pd.date_range(start, pd.Timestamp.today().strftime("%Y-%m-01"), freq="M")


[WARN] CSV не найдены → использую синтетические ряды для демонстрации.
[OK] Dataset saved → outputs/ust_infl_ffr_dataset.csv
[OK] Static correlations saved.
[DONE] Все таблицы и графики сохранены в ./outputs
   infl_vs_y5_level  infl_vs_y30_level  infl_diff_vs_y5  infl_diff_vs_y30  \
0             0.236              0.224             0.17             0.181   

   fed_diff_vs_y5  fed_diff_vs_y30  slope_30_5_vs_infl  \
0           0.151            0.153                0.08   

   right_minus_left_infl_level  right_minus_left_infl_diff  \
0                       -0.013                       0.011   

   right_minus_left_fed_diff  
0                      0.001  


Что получишь после запуска

В папке ./outputs будут:

ust_infl_ffr_dataset.csv (собранный датасет уровней, month-end)

corr_*.csv, rolling_corr_*.csv, crosscorr_*.csv

inflation_shock_windows.csv, rate_shock_windows.csv

approx_returns.csv и correlations_shock_windows.csv

PNG-графики: rolling/cross-corr, линейные с шейдингом, scatter-диаграммы

Как подставить реальные данные

Положи рядом со скриптом:

inflation_expectations.csv — date, infl_exp (или value/breakeven/expectations)

treasury_yields.csv — date, y5, y10, y20, y30 (или DGS5,DGS10,DGS20,DGS30)

fed_funds.csv — date, fed_funds (или DFF/EFFR/fedrate/ffr)

Скрипт сам:

приведёт к month-end,

найдёт shock-периоды по 95-му перцентилю |Δ|,

построит все графики и корреляции,

сравнит правый vs левый край (30Y против 5Y) и наклон slope_30_5.

Если нужно — докину блоки OLS (Newey–West) и/или расчёт total return пар-бондов с купоном и convexity.